In [ ]:
pip install colorama

In [ ]:

import torch
from torchvision import datasets, transforms
import tensorflow as tf
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import random
import math
import sys
from enum import Enum
from colorama import Fore
import csv
import copy
import os
from itertools import chain
from tensorflow.keras.optimizers import Adam
from PIL import Image
import numpy as np
from IPython.display import display
import cProfile
import pstats

In [ ]:
#무적의 상수
PRINT_FLAG = False
NUM_OF_SKILL = 4
NUMBER_OF_CASES_HP = 20
NUM_OF_STATES = 9

In [ ]:
# ENUM
class Enum(Enum):
  # 상성별 index 저장한 DICTIONARY다 하하하하
    TYPE_DICT = {
        "노말": 0,
        "격투": 1,
        "독": 2,
        "땅": 3,
        "비행": 4,
        "벌레": 5,
        "바위": 6,
        "고스트": 7,
        "강철": 8,
        "불꽃": 9,
        "물": 10,
        "전기": 11,
        "풀": 12,
        "얼음": 13,
        "에스퍼": 14,
        "드래곤": 15,
        "악": 16
    }
  #상성표! 여기 오류가 있다면 난 자살하고 말거야
    COMPATIBILITY_TABLE = [
        [1, 1, 1, 1, 1, 1, 0.5, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], #노말
        [2, 1, 0.5, 1, 0.5, 0.5, 2, 0, 2, 1, 1, 1, 1, 2, 0.5, 1, 2], #격투
        [1, 1, 0.5, 0.5, 1, 1, 0.5, 0.5, 0, 1, 1, 1, 2, 1, 1, 1, 1], # 독
        [1, 1, 2, 1, 0, 0.5, 2, 1, 2, 2, 1, 2, 0.5, 1, 1, 1, 1], #땅
        [1, 2, 1, 1, 1, 2, 0.5, 1, 0.5, 1, 1, 0.5, 2, 1, 1, 1, 1], #비행
        [1, 0.5, 0.5, 1, 0.5, 1, 1, 0.5, 0.5, 0.5, 1, 1, 2, 1, 2, 1, 2], #벌레
        [1, 0.5, 1, 0.5, 2, 2, 1, 1, 0.5, 2, 1, 1, 1, 2, 1, 1, 1], #바위
        [0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0.5], #고스트
        [1, 1, 1, 1, 1, 1, 2, 1, 0.5, 0.5, 0.5, 0.5, 1, 2, 1, 1, 1], #강철
        [1, 1, 1, 1, 1, 2, 0.5, 1, 2, 0.5, 0.5, 1, 2, 2, 1, 0.5, 1], #불꽃
        [1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 0.5, 1, 0.5, 1, 1, 0.5, 1], #물
        [1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 2, 0.5, 0.5, 1, 1, 0.5, 1], #전기
        [1, 1, 0.5, 2, 0.5, 0.5, 2, 1, 0.5, 0.5, 2, 1, 0.5, 1, 1, 0.5, 1], #풀
        [1, 1, 1, 2, 2, 1, 1, 1, 0.5, 0.5, 0.5, 1, 2, 0.5, 1, 2, 1], #얼음
        [1, 2, 2, 1, 1, 1, 1, 1, 0.5, 1, 1, 1, 1, 1, 0.5, 1, 0], #에스퍼
        [1, 1, 1, 1, 1, 1, 1, 1, 0.5, 1, 1, 1, 1, 1, 1, 2, 1], #드래곤
        [1, 0.5, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0.5] #악
    ]
  #선공기 예외처리
    PRIORITY_SKILL= ["신속", "마하펀치", "아쿠아제트", "전광석화", "기습", "얼음뭉치", "야습"]

In [ ]:
#포켓몬 기술 객체
class PokemonSkill:
    def __init__(self, name, power, accuracy, type, pp, attack_type):
        self.name = name
        self.power = power
        self.accuracy = accuracy
        self.type = type
        self.pp = pp
        self.maxpp = pp
        self.attack_type = attack_type
    def reset(self):
        self.pp = self.maxpp

In [ ]:

class Pokemon:
    def __init__(self, name, types_list, ev, iv, health, attack, defense, special_attack, special_defense, speed, skill_list):
        self.name = name #이름
        self.types_list = types_list #타입
        if (ev == -1): #노력치
            ev = 63#random.randint(0, 63) #노력치 랜덤
        else:
            self.ev = ev # 노력치 지정
        if (iv == -1): #개체치
            iv = 31#random.randint(0, 31) #개체치 랜덤
        else:
            self.iv = iv # 개체치 지정
        self.health = math.floor(health + (ev / 4 + iv) / 2 + 110) #체력
        self.maxhealth = self.health
        self.attack = math.floor(attack + (ev / 4 + iv) / 2 + 5) #공격
        self.defense = math.floor(defense + (ev / 4 + iv) / 2 + 5) #방어
        self.special_attack = math.floor(special_attack + (ev / 4 + iv) / 2 + 5) #특수공격
        self.special_defense = math.floor(special_defense + (ev / 4 + iv) / 2 + 5) #특수방어
        self.speed = math.floor(speed + (ev / 4 + iv) / 2 + 5) #스피드
        self.hitrate = 1
        self.skill_list = skill_list #기술 리스트
        self.skill_probabilities = [[[0.25 for _ in range(NUM_OF_SKILL)] for _ in range(NUMBER_OF_CASES_HP)] for _ in range(NUMBER_OF_CASES_HP)]# for _ in range[NUM_OF_STATES]]#핵심 학습 대상
        self.state = []
        self.original_values = {
            'name': name,
            'types_list': types_list,
            'ev': ev,
            'iv': iv,
            'health': health,
            'attack': attack,
            'defense': defense,
            'special_attack': special_attack,
            'special_defense': special_defense,
            'speed': speed,
            'skill_list': skill_list
        }
    def reset_attributes(self):
        self.name = self.original_values['name']
        self.types_list = self.original_values['types_list']
        self.ev = self.original_values['ev']
        self.iv = self.original_values['iv']
        self.health = math.floor(self.original_values['health'] + (self.ev / 4 + self.iv) / 2 + 110)
        self.maxhealth = self.health
        self.attack = math.floor(self.original_values['attack'] + (self.ev / 4 + self.iv) / 2 + 5)
        self.defense = math.floor(self.original_values['defense'] + (self.ev / 4 + self.iv) / 2 + 5)
        self.special_attack = math.floor(self.original_values['special_attack'] + (self.ev / 4 + self.iv) / 2 + 5)
        self.special_defense = math.floor(self.original_values['special_defense'] + (self.ev / 4 + self.iv) / 2 + 5)
        self.speed = math.floor(self.original_values['speed'] + (self.ev / 4 + self.iv) / 2 + 5)
        self.hitrate = 1
        self.skill_list = self.original_values['skill_list']
        for i in self.skill_list:
          i.reset()
        self.state = []
    #정보 output
    def display_info(self):
        print(f"Name: {self.name}")
        print(f"Type: {self.types_list}")
        print(f"Health: {self.maxhealth}")

    def check_stab(self, t_list, t): #자속 보정 확인
        if (t in t_list):
            return 1.5
        else:
            return 1

    def check_weather(self, weather, type):
        if (weather == "햇살"):
            if (type == "불꽃"):
                return 1.5
            elif (type == "풀"):
                return 1.2
            elif (type == "물"):
                return 0.5
        if (weather == "비"):
            if (type == "물"):
                return 1.5
            elif (type == "전기"):
                return 1.2
            elif (type == "불꽃"):
                return 0.5
        return 1

    #반동 계산식
    def take_reaction_damage(self, skill, rate):
        oldhealth = self.health
        self.health -= math.floor((self.attack * skill.power / (self.maxhealth * self.defense) * 100) / rate)
        if PRINT_FLAG:
          print(f"{self.name}은(는) {oldhealth - self.health}의 반동을 받았다! 현재 체력: {self.health}/{self.maxhealth}")

    def special_skill(self, skill, weather):
        skill_result_list = [0, skill, "정상"]
        temp_repitition = 1
        #랭크업
        if (skill.name == "칼춤"):
            self.attack = math.floor(self.attack * 1.25)
            if (PRINT_FLAG):
                print(f"{self.name}의 공격이 올랐다!")
        if (skill.name in ["용의춤", "정리정돈"]):
            self.attack = math.floor(self.attack * 1.12)
            self.speed = math.floor(self.speed * 1.12)
            if (PRINT_FLAG):
                print(f"{self.name}의 공격과 스피드가 올랐다!")
        elif (skill.name == "나쁜음모"):
            self.special_attack = math.floor(self.special_attack * 1.25)
            if (PRINT_FLAG):
                print(f"{self.name}의 특수공격이 올랐다!")
        elif (skill.name in ["방어지령", "비축하기"]):
            self.defense = math.floor(self.defense * 1.12)
            self.special_defense = math.floor(self.special_defense * 1.12)
            if (PRINT_FLAG):
                print(f"{self.name}의 방어와 특수방어가 올랐다!")
        elif (skill.name in ["나비춤"]):
            self.defense = math.floor(self.defense * 1.12)
            self.special_defense = math.floor(self.special_defense * 1.12)
            self.speed = math.floor(self.speed * 1.12)
            if (PRINT_FLAG):
                print(f"{self.name}의 방어와 특수방어, 그리고 스피드가 올랐다!")
        elif (skill.name in ["은빛바람", "괴상한바람", "원시의힘"]):
            if (random.randint(1, 100) <= 10):
                self.attack = math.floor(self.attack * 1.12)
                self.special_attack = math.floor(self.special_attack * 1.12)
                self.defense = math.floor(self.defense * 1.12)
                self.special_defense = math.floor(self.special_defense * 1.12)
                self.speed = math.floor(self.speed * 1.12)
                if (PRINT_FLAG):
                    print(f"{self.name}의 능력치가 올랐다!")
        elif (skill.name in ["코멧펀치"]):
            if (random.randint(1, 100) <= 20):
                self.attack = math.floor(self.attack * 1.12)
                if (PRINT_FLAG):
                    print(f"{self.name}의 공격이 올랐다!")
        elif (skill.name in ["메탈클로"]):
            if (random.randint(1, 100) <= 10):
                self.attack = math.floor(self.attack * 1.12)
                if (PRINT_FLAG):
                    print(f"{self.name}의 공격이 올랐다!")
        elif (skill.name in ["강철날개"]):
            if (random.randint(1, 100) <= 10):
                self.defense = math.floor(self.defense * 1.12)
                if (PRINT_FLAG):
                    print(f"{self.name}의 방어가 올랐다!")
        elif (skill.name in ["배북"]):
                self.health = math.floor(self.health * 0.5)
                self.attack = math.floor(self.attack * 1.5)
                if (PRINT_FLAG):
                    print(f"{self.name}은 체력을 깎아서 풀 파워로 만들었다! 현재 체력: {self.health}/{self.maxhealth}")
        elif (skill.name in ["고속이동", "록커트", "순풍"]):
            self.speed = math.floor(self.speed * 1.25)
            if (PRINT_FLAG):
                print(f"{self.name}의 스피드가 올랐다!")
        elif (skill.name in ["철벽", "녹기"]):
            self.defense = math.floor(self.defense * 1.25)
            if (PRINT_FLAG):
                print(f"{self.name}의 방어가 올랐다!")
        elif (skill.name in ["배리어러시"]):
            self.defense = math.floor(self.defense * 1.12)
            if (PRINT_FLAG):
                print(f"{self.name}의 방어가 올랐다!")
        elif (skill.name in ["망각술"]):
            self.special_defense = math.floor(self.special_defense * 1.25)
            if (PRINT_FLAG):
                print(f"{self.name}의 특수방어가 올랐다!")
        elif (skill.name in ["안개제거"]):
            self.hitrate = math.floor(self.hitrate * 1.12)
            if (PRINT_FLAG):
                print(f"{self.name}의 명중률이 올랐다!")
        elif (skill.name in ["벌크업"]):
            self.attack = math.floor(self.attack * 1.12)
            self.defense = math.floor(self.defense * 1.12)
            if (PRINT_FLAG):
                print(f"{self.name}의 공격과 방어가 올랐다!")

        #회복기
        elif (skill.name in ["게으름피우기", "회복지령", "알낳기", "HP회복", "아침햇살", "달빛", "우유마시기"]):
            oldhealth = self.health
            if (skill.name in ["아침햇살", "달빛"] and weather[0] != ""):
                self.health += self.maxhealth
            else:
                self.health += math.floor(self.health * 0.5)
            if (self.health > self.maxhealth):
                self.health = self.maxhealth
            if (self.health - oldhealth > 0):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) {self.health - oldhealth}만큼의 체력을 회복했다! 현재 체력: {self.health}/{self.maxhealth}")
            else:
                if (PRINT_FLAG):
                    print(f"그러나 실패했다!")
        elif (skill.name in ["꿀꺽"]):
            oldhealth = self.health
            self.health += math.floor(self.health * 0.35)
            if (self.health > self.maxhealth):
                self.health = self.maxhealth
            if (self.health - oldhealth > 0):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) {self.health - oldhealth}만큼의 체력을 회복했다! 현재 체력: {self.health}/{self.maxhealth}")
            else:
                if (PRINT_FLAG):
                    print(f"그러나 실패했다!")

        #랭크다운
        elif (skill.name == "인파이트"):
            self.defense = math.floor(self.defense * 0.85)
            self.special_defense = math.floor(self.special_defense * 0.85)
            if (PRINT_FLAG):
                print(f"{self.name}의 방어와 특수방어가 떨어졌다!")
        elif (skill.name == "암해머"):
            self.speed = math.floor(self.speed * 0.85)
            if (PRINT_FLAG):
                print(f"{self.name}의 스피드가 떨어졌다!")
        elif (skill.name in ["용성군", "리프스톰", "오버히트"]):
            self.special_attack = math.floor(self.special_attack * 0.75)
            if (PRINT_FLAG):
                print(f"{self.name}의 특수공격이 크게 떨어졌다!")
        elif (skill.name in ["엄청난힘"]):
            self.attack = math.floor(self.attack * 0.85)
            self.defense = math.floor(self.defense * 0.85)
            if (PRINT_FLAG):
                print(f"{self.name}의 공격과 방어가 떨어졌다!")

        #날씨
        elif (skill.name == "쾌청"):
            if (weather[0] != "햇살"):
                weather[0] = "햇살"
                weather[1] = 3
                if (PRINT_FLAG):
                    print(f"햇살이 강해졌다!")
            else:
                if (PRINT_FLAG):
                    print(f"그러나 실패했다!")
        elif (skill.name == "비바라기"):
            if (weather[0] != "비"):
                weather[0] = "비"
                weather[1] = 3
                if (PRINT_FLAG):
                    print(f"비가 내리기 시작했다!")
            else:
                if (PRINT_FLAG):
                    print(f"그러나 실패했다!")
        elif (skill.name == "싸라기눈"):
            if (weather[0] != "눈"):
                weather[0] = "눈"
                weather[1] = 3
                if (PRINT_FLAG):
                    print(f"눈이 내리기 시작했다!")
            else:
                if (PRINT_FLAG):
                    print(f"그러나 실패했다!")

        #충전기
        elif (skill.name == "솔라빔"):
            if (weather[0] != "햇살"):
                if ("충전" not in self.state):
                    skill.pp += 1
                    if (PRINT_FLAG):
                        sys.stdout.write("\033[F")
                        sys.stdout.write("\033[K")
                        print(f"{self.name}은(는) 빛을 모으고 있다.")
                    skill_result_list[0] = -1
                    skill_result_list[2] = "충전"
                    self.state.append("충전")
                else:
                    self.state.remove("충전")
        elif (skill.name in ["공중날기", "불새", "뛰어오르기"]):
            if ("공중" not in self.state):
                skill.pp += 1
                if (PRINT_FLAG):
                    sys.stdout.write("\033[F")
                    sys.stdout.write("\033[K")
                    print(f"{self.name}은(는) 하늘로 날아올랐다!")
                skill_result_list[0] = -1
                self.state.append("공중")
            else:
                self.state.remove("공중")
                if (skill.name == "불새"):
                    if (random.randint(1, 100) <= 30):
                        skill_result_list[2] = "풀죽음"
                if (skill.name == "뛰어오르기"):
                    if (random.randint(1, 100) <= 30):
                        skill_result_list[2] = "마비"
        elif (skill.name in ["구멍파기"]):
            if ("지하" not in self.state):
                skill.pp += 1
                if (PRINT_FLAG):
                    sys.stdout.write("\033[F")
                    sys.stdout.write("\033[K")
                    print(f"{self.name}은(는) 땅속으로 파고들었다!")
                skill_result_list[0] = -1
                self.state.append("지하")
            else:
                self.state.remove("지하")

        #반동기
        elif (skill.name in ["와일드볼트", "발버둥"]):
            self.take_reaction_damage(skill, 4)
        elif (skill.name in ["우드해머", "이판사판태클", "브레이브버드", "웨이브태클", "볼트태클"]):
            self.take_reaction_damage(skill, 3)
        elif (skill.name in ["양날박치기", "철제광선"]):
            self.take_reaction_damage(skill, 2)

        #즉사기
        elif (skill.name in ["절대영도", "뿔드릴"]):
            skill_result_list[2] = "즉사"

        #수면기
        elif (skill.name == "하품"):
            if (PRINT_FLAG):
                print(f"{self.name}은(는) 상대의 졸음을 유도했다!")
            skill_result_list[2] = "졸림"
        elif (skill.name in ["최면술", "버섯포자", "노래하기", "수면가루", "악마의키스"]):
            skill_result_list[2] = "졸음1"

        #얼음
        elif (skill.name  in ["얼음엄니", "냉동펀치", "냉동빔", "눈보라"]):
            if (random.randint(1, 100) <= 10):
                skill_result_list[2] = "얼음"

        #화상
        elif (skill.name == "플레어드라이브"):
            self.take_reaction_damage(skill, 3, PRINT_FLAG)
            if (random.randint(1, 100) <= 10):
                skill_result_list[2] = "화상"
        elif (skill.name in ["불꽃펀치", "화염방사", "불대문자", "열풍"]):
            if (random.randint(1, 100) <= 10):
                skill_result_list[2] = "화상"
        elif (skill.name in ["열탕", "분연"]):
            if (random.randint(1, 100) <= 30):
                skill_result_list[2] = "화상"
        elif (skill.name in ["트라이어택"]):
            if (random.randint(1, 100) <= 8):
                skill_result_list[2] = "마비"
            elif (random.randint(1, 100) <= 8):
                skill_result_list[2] = "화상"
            elif (random.randint(1, 100) <= 8):
                skill_result_list[2] = "얼음"
        elif (skill.name in ["불꽃엄니"]):
            if (random.randint(1, 100) <= 10):
                skill_result_list[2] = "화상"
            elif (random.randint(1, 100) <= 15):
                skill_result_list[2] = "풀죽음"
        elif (skill.name in ["블레이즈킥"]):
            if (random.randint(1, 100) <= 10):
                skill_result_list[2] = "화상"
            elif (random.randint(1, 100) <= 15):
                skill_result_list[2] = "급소증가"
        elif (skill.name in ["도깨비불"]):
            skill_result_list[2] = "화상"

        #마비
        elif (skill.name in ["번개펀치", "10만볼트", "번개엄니"]):
            if (random.randint(1, 100) <= 10):
                skill_result_list[2] = "마비"
        elif (skill.name in ["방전", "번개", "누르기", "스파크"]):
            if (random.randint(1, 100) <= 30):
                skill_result_list[2] = "마비"
        elif (skill.name in ["핥기"]):
            if (random.randint(1, 100) <= 50):
                skill_result_list[2] = "마비"
        elif (skill.name in ["볼부비부비", "전기자석파", "뱀눈초리", "저리가루"]):
            skill_result_list[2] = "마비"

        #혼란
        elif (skill.name in ["시그널빔", "환상빔"]):
            if (random.randint(1, 100) <= 10):
                skill_result_list[2] = "혼란"
        elif (skill.name in ["록클라임", "물의파동", "잼잼펀치"]):
            if (random.randint(1, 100) <= 20):
                skill_result_list[2] = "혼란"
        elif (skill.name in ["폭풍"]):
            if (random.randint(1, 100) <= 30):
                skill_result_list[2] = "혼란"
        elif (skill.name in ["수다", "이상한빛", "흔들흔들댄스", "천사의키스", "폭발펀치"]):
            skill_result_list[2] = "혼란"

        #중독
        elif (skill.name in ["오물폭탄", "독찌르기", "더스트슈트"]):
            if (random.randint(1, 100) <= 30):
                skill_result_list[2] = "중독"
        elif (skill.name in ["크로스포이즌", "포이즌테일", "오물웨이브"]):
            if (random.randint(1, 100) <= 10):
                skill_result_list[2] = "중독"
            elif (random.randint(1, 100) <= 10):
                skill_result_list[2] = "급소증가"
        elif (skill.name == "맹독"):
            skill_result_list[2] = "중독"

        #풀죽음
        elif (skill.name in ["사념의박치기", "폭포오르기", "악의파동"]):
            if (random.randint(1, 100) <= 20):
                skill_result_list[2] = "풀죽음"
        elif (skill.name in ["스톤샤워", "아이언헤드", "에어슬래시", "짓밟기", "고드름떨구기"]):
            if (random.randint(1, 100) <= 30):
                skill_result_list[2] = "풀죽음"
        elif (skill.name in ["안다리걸기"]):
            if (random.randint(1, 100) <= 50):
                skill_result_list[2] = "풀죽음"
        elif (skill.name in ["속이기"]):
            skill_result_list[2] = "풀죽음"

        #연속기
        elif (skill.name in ["손바닥치기", "록블라스트"]):
            temp_repitition = random.randint(2, 5)
            if (PRINT_FLAG):
                print(f"{temp_repitition}번 썼다!")

        #급소확률증가
        elif (skill.name in ["스톤에지", "공격지령", "섀도클로", "깜짝베기", "에어컷터", "사이코컷터", "리프블레이드"]):
            skill_result_list[2] = "급소증가"

        #공깎
        elif (skill.name == "애교부리기"):
            skill_result_list[2] = "크게공깎"

        #공깎
        elif (skill.name == "벌레의저항"):
            skill_result_list[2] = "특공깎"

        #방깎
        elif (skill.name in ["깨물어부수기", "아쿠아브레이크"]):
            if (random.randint(1, 100) <= 20):
                skill_result_list[2] = "방깎"
        elif (skill.name == "아이언테일"):
            if (random.randint(1, 100) <= 30):
                skill_result_list[2] = "방깎"
        elif (skill.name == "브레이크클로"):
            if (random.randint(1, 100) <= 50):
                skill_result_list[2] = "방깎"
        elif (skill.name == "싫은소리"):
                skill_result_list[2] = "크게방깎"

        #스피드하락
        elif (skill.name in ["얼어붙은바람", "암석봉인"]):
            skill_result_list[2] = "스피드하락"

        #특방깎
        elif (skill.name in ["러스터캐논", "사이코키네시스", "대지의힘", "에너지볼", "기합구슬"]):
            if (random.randint(1, 100) <= 10):
                skill_result_list[2] = "특방깎"
        elif (skill.name == "섀도볼"):
            if (random.randint(1, 100) <= 20):
                skill_result_list[2] = "특방깎"

        #명중률
        elif (skill.name in ["대포무노포"]):
            if (random.randint(1, 100) <= 50):
                skill_result_list[2] = "명중률하락"
        elif (skill.name in ["탁류", "진흙폭탄"]):
            if (random.randint(1, 100) <= 30):
                skill_result_list[2] = "명중률하락"
        elif (skill.name in ["플래시", "진흙뿌리기", "작아지기", "숟가락휘기"]):
            skill_result_list[2] = "명중률하락"

        #정화
        elif (skill.name in ["치료방울"]):
            if (len(self.state) > 0):
                self.state = []
            else:
                if (PRINT_FLAG):
                    print(f"그러나 실패했다!")

        #반동
        elif (skill.name in ["파괴광선", "기가임팩트"]):
            self.state.append("반동1")

        #난동기
        elif (skill.name in ["꽃잎댄스", "역린", "난동부리기"]):
            if ("혼란" in self.state):
                if (PRINT_FLAG):
                    print(f"그러나 실패했다!")
                skill_result_list[2] = "빗나감"
            if (not lists_have_common_values(self.state, ["난동1", "난동2", "난동3", "혼란"])):
                self.state.append("난동1")

        #흡혈기
        elif (skill.name in ["기가드레인", "드레인펀치"]):
            skill_result_list[2] = self

        #체력비례기술
        elif (skill.name == "분노의앞니"):
            skill_result_list[2] = "50퍼"

        #속도비례기술
        elif (skill.name == "자이로볼"):
            skill_result_list[2] = self
            skill_result_list[0] = -1
        elif (skill.name == "보복"):
            skill_result_list[2] = self
            skill_result_list[0] = -2

        #멸망의노래
        elif (skill.name == "멸망의노래"):
            skill_result_list[2] = "멸망"

        #카운터
        elif (skill.name == "카운터"):
            skill_result_list[2] = "카운터"
        elif (skill.name == "미러코트"):
            skill_result_list[2] = "미러코트"

        #잠재파워
        elif (skill.name == "잠재파워"):
            skill_result_list[1].type = self.value_mapping(int(self.health % 16))

        #스탯변경
        elif (skill.name == "파워트릭"):
            temp = self.attack
            self.attack = self.defense
            self.defense = temp
            if (PRINT_FLAG):
                print(f"{self.name}의 공격과 방어가 교환됬다!")

        #특성추가
        elif (skill.name == "숲의저주"):
            skill_result_list[2] = "풀추가"

        #특성제거
        elif (skill.name == "불사르기"):
            if ("얼음" in self.state):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 전신의 불꽃을 모두 태워서 얼음 상태를 벗어났다!")
                self.state.remove("얼음")
            if ("불꽃" in self.types_list):
                self.types_list.remove("불꽃")
        elif (skill.name == "미러코트"):
            skill_result_list[2] = "미러코트"

        #객기
        elif (skill.name == "객기"):
            if (lists_have_common_values(self.state, ["마비", "화상", "독"])):
                skill.power = 150
            else:
                skill.power = 70

        #상태이상면역
        elif (skill.name == "신비의부적"):
            if ("신비" not in self.state):
                if (PRINT_FLAG):
                    print(f"{self.name}은 신비의부적의 보호를 받기 시작했다!")
                self.state.append("신비")
            else:
                if (PRINT_FLAG):
                    print(f"그러나 실패했다!")

        #변신
        elif (skill.name == "변신"):
            skill_result_list[2] = self

        if (skill.power > 0 and not lists_have_common_values(self.state, ["공중", "충전", "지하"])):
            skill_result_list[0] = math.floor((self.attack if skill.attack_type == "물리" else self.special_attack) * skill.power * temp_repitition * self.check_stab(self.types_list, skill.type) * self.check_weather(weather[0], skill.type))

        return skill_result_list

    def use_skill(self, index, weather):

        current_skill = self.skill_list[index]

        self.skill_list[index].pp -= 1
        if PRINT_FLAG:
          print(f"{self.name}의 {current_skill.name}! (남은 PP {self.skill_list[index].pp}/{self.skill_list[index].maxpp})")

        if (random.randint(1, 100) <= current_skill.accuracy * self.hitrate):
            return self.special_skill(current_skill, weather)

        else:
            if PRINT_FLAG:
              print(f"그러나 빗나가고 말았다!")
            return [0, current_skill, "빗나감"]

    def take_damage(self, damage):
        #상성 무효인지 확인
        if (damage[2] == "빗나감"):
            return
        if ("공중" in self.state and damage[0] != 0):
            if (damage[1].name != "스카이어퍼"):
                if PRINT_FLAG:
                  print(f"그러나 빗나가고 말았다!")
                return
        if ("지하" in self.state and damage[0] != 0):
            if (damage[1].name != "지진"):
                if (PRINT_FLAG):
                    print(f"그러나 빗나가고 말았다!")
                return
        if (self.compatibility_check(damage[1].type, self.types_list) == 0 and damage[0] > 0):
            if PRINT_FLAG:
              print(f"{damage[1].name}은(는) {self.name}에게 효과가 없는 듯 하다...")
            return


        oldhealth = self.health
        crit = random.randint(85, 100)
        heal = False

        #상태이상 체크
        if (damage[2] == "졸림"):
            for state in self.state:
                if (state in ["피곤", "졸음1", "졸음2", "졸음3", "졸음4"]):
                    if (PRINT_FLAG):
                        sys.stdout.write("\033[F")
                        sys.stdout.write("\033[K")
                        print(f"그러나 실패했다!")
                    return
            self.state.append("피곤")
        elif (damage[2] == "졸음1"):
            for state in self.state:
                if (state in ["피곤", "졸음1", "졸음2", "졸음3", "졸음4"]):
                    if (PRINT_FLAG):
                        print(f"그러나 실패했다!")
                    return
            self.state.append("졸음1")

        elif (damage[2] == "얼음"):
            if ("얼음" not in self.state):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 얼어붙었다!")
                self.state.append("얼음")
            if ("화상" in self.state):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 얼음으로 인해 화상이 나았다!")
                self.state.remove("화상")
                self.attack = math.floor(self.attack * 2)

        elif (damage[2] == "화상"):
            if ("화상" not in self.state):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 화상을 입었다!")
                self.attack = math.floor(self.attack / 2)
                self.state.append("화상")
            else:
                if(damage[1].name == "도깨비불"):
                    if (PRINT_FLAG):
                        print(f"그러나 실패했다!")
            if ("얼음" in self.state):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 화상으로 인해 얼음 상태가 풀렸다!")
                self.state.remove("얼음")

        elif (damage[2] == "마비"):
            if ("마비" not in self.state and "전신마비" not in self.state):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 마비되어 기술이 나오기 어려워졌다!")
                self.speed = math.floor(self.speed / 2)
                self.state.append("마비")
            else:
                if (damage[1].name in ["전기자석파", "뱀눈초리", "저리가루"]):
                    if (PRINT_FLAG):
                        print(f"그러나 실패했다!")


        elif (damage[2] == "혼란"):
            if ("혼란" not in self.state and "자해" not in self.state):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 혼란에 빠졌다!")
                self.state.append("혼란")
            else:
                if (damage[1].name in ["이상한빛","흔들흔들댄스", "천사의키스"]):
                    if (PRINT_FLAG):
                        print(f"그러나 실패했다!")

        elif (damage[2] == "중독"):
            if ("중독" not in self.state):
                if (PRINT_FLAG):
                    print(f"{self.name}의 몸에 독이 퍼졌다!")
                self.state.append("중독")
            else:
                if (damage[1].name == "맹독"):
                    if (PRINT_FLAG):
                        print(f"그러나 실패했다!")

        elif (damage[2] == "풀죽음"):
            if ("풀죽음1" not in self.state):
                self.state.append("풀죽음1")

        elif (damage[2] == "멸망"):
            if (not lists_have_common_values(["멸망3", "멸망2", "멸망1", "멸망0"], self.state)):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 멸망의 노래를 들었다!")
                self.state.append("멸망3")
            else:
                if (PRINT_FLAG):
                    print(f"그러나 실패했다!")

        elif (damage[2] == "급소증가"):
            crit += 5
            if (crit > 100):
                crit = 100

        elif (damage[2] == "크게공깎"):
            if (PRINT_FLAG):
                print(f"{self.name}의 공격이 크게 떨어졌다!")
            self.attack = math.floor(self.attack * 0.75)

        elif (damage[2] == "방깎"):
            if (PRINT_FLAG):
                print(f"{self.name}의 방어가 떨어졌다!")
            self.defense = math.floor(self.defense * 0.85)

        elif (damage[2] == "크게방깎"):
            if (PRINT_FLAG):
                print(f"{self.name}의 방어가 크게 떨어졌다!")
            self.defense = math.floor(self.defense * 0.75)

        elif (damage[2] == "스피드하락"):
            if (PRINT_FLAG):
                print(f"{self.name}의 스피드가 떨어졌다!")
            self.speed = math.floor(self.speed * 0.85)

        elif (damage[2] == "특공깎"):
            if (PRINT_FLAG):
                print(f"{self.name}의 특수공격이 떨어졌다!")
            self.special_attack = math.floor(self.special_attack * 0.85)

        elif (damage[2] == "특방깎"):
            if (PRINT_FLAG):
                print(f"{self.name}의 특수방어가 떨어졌다!")
            self.special_defense = math.floor(self.special_defense * 0.85)

        elif (damage[2] == "명중률하락"):
            if (PRINT_FLAG):
                print(f"{self.name}의 명중률이 떨어졌다!")
            self.hitrate = math.floor(self.special_defense * 0.9)

        elif (damage[2] == "50퍼"):
            self.health -= math.ceil(self.health / 2)

        elif (damage[2] == "카운터"):
            self.health -= math.floor((self.attack * 100 / (self.maxhealth * self.defense / 0.611) * crit))

        elif (damage[2] == "미러코트"):
            self.health -= math.floor((self.special_attack * 100 / (self.maxhealth * self.special_defense / 0.611) * crit))

        elif (damage[2] == "즉사"):
            if (PRINT_FLAG):
                print(f"일격필살!")
            self.health = 0

        elif (damage[2] == "풀추가"):
            if ("풀" not in self.types_list):
                self.types_list.append("풀")
                if (PRINT_FLAG):
                    print(f"풀의저주로 인해 {self.name}에게 풀 속성이 추가되었다!")
            else:
                if (PRINT_FLAG):
                    print(f"그러나 실패했다!")

        elif (type(damage[2]) is Pokemon and damage[0] == -1): #자이로볼
            temp_power = math.floor(25 * (self.speed / (damage[2].speed + 1)))
            if (temp_power > 150):
                temp_power = 150
            damage[0] = math.floor((damage[2].attack if damage[1].attack_type == "물리" else damage[2].special_attack) * temp_power * self.check_stab(damage[2].types_list, damage[1].type))

        elif (type(damage[2]) is Pokemon and damage[0] == -2): #보복
            if (damage[2].speed < self.speed):
                temp_power = 100
            else:
                temp_power = 50
            damage[0] = math.floor((damage[2].attack if damage[1].attack_type == "물리" else damage[2].special_attack) * temp_power * self.check_stab(damage[2].types_list, damage[1].type))

        elif (type(damage[2]) is Pokemon and damage[0] == 0): #변신
            if (PRINT_FLAG):
                print(f"{damage[2].name}은(는) {self.name}으로 변신했다!")
            damage[2] = copy.deepcopy(self)

        elif (type(damage[2]) is Pokemon):
            heal = True


        if (crit == 100):
            crit = 150
        if (damage[1].attack_type == "물리" and damage[0] > 0):
            self.health -= math.floor((damage[0] / (self.maxhealth * self.defense / 0.611) * crit) * self.compatibility_check(damage[1].type, self.types_list))
        elif (damage[1].attack_type == "특수" and damage[0] > 0):
            self.health -= math.floor((damage[0] / (self.maxhealth * self.special_defense / 0.611) * crit) * self.compatibility_check(damage[1].type, self.types_list))

        if (oldhealth != self.health):
            if (damage[2] not in ["50퍼", "카운터", "미러코트"]):
                if (crit == 150):
                    if (PRINT_FLAG):
                        print("급소에 맞았다!")
                if (self.compatibility_check(damage[1].type, self.types_list) > 1):
                    if (PRINT_FLAG):
                        print(f"효과가 굉장했다!")
                elif (self.compatibility_check(damage[1].type, self.types_list) < 1):
                    if (PRINT_FLAG):
                        print(f"효과가 별로인듯 하다...")
            if (PRINT_FLAG):
                print(f"{self.name}은(는) {oldhealth - self.health}의 데미지를 받았다! 현재 체력: {self.health if self.health >= 0 else 0}/{self.maxhealth}")

        if (type(damage[2]) == Pokemon and heal):
            damage_taken = oldhealth - self.health
            oldhealth = damage[2].health
            damage[2].health += math.floor((damage_taken) / 2)
            if (damage[2].health > damage[2].maxhealth):
                damage[2].health = damage[2].maxhealth
            if (PRINT_FLAG):
                print(f"{damage[2].name}은(는) {damage[2].health - oldhealth}의 체력을 회복했다! 현재 체력: {damage[2].health}/{damage[2].maxhealth}")

    def state_control(self):
        #Flag 초기화
        self.state = ["마비" if i == "전신마비" else i for i in self.state]
        self.state = ["혼란" if i == "자해" else i for i in self.state]
        self.state.sort(reverse = True)
        if ("신비" in self.state and len(self.state) > 1):
            if (PRINT_FLAG):
                print(f"{self.name}은(는) 신비의부적으로 상태이상을 이겨냈다!")
            self.state = []

        for i in range(len(self.state)):

            #멸망 코드
            if (self.state[i] == "멸망3"):
                if (PRINT_FLAG):
                    print(f"{self.name}의 남은 턴 3!")
                self.state[i] = "멸망2"
            elif (self.state[i] == "멸망2"):
                if (PRINT_FLAG):
                    print(f"{self.name}의 남은 턴 2!")
                self.state[i] = "멸망1"
            elif (self.state[i] == "멸망1"):
                if (PRINT_FLAG):
                    print(f"{self.name}의 남은 턴 1!")
                self.state[i] = "멸망0"
            elif (self.state[i] == "멸망0"):
                if (PRINT_FLAG):
                    print(f"{self.name}의 남은 턴 0!")
                self.health = 0
                self.state[i] = "멸망"
                return

            #잠듦 코드
            if (self.state[i] == "피곤"):
                self.state[i] = "졸음1"
            elif (self.state[i] == "졸음1"):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 잠들어 버렸다!")
                self.state[i] = "졸음2"
            elif (self.state[i] == "졸음2"):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 쿨쿨 잠들어 있다.")
                if (random.randint(1, 100) <= 33):
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 잠에서 깨어났다!")
                    self.state[i] = "정상"
                else:
                    self.state[i] = "졸음3"
            elif (self.state[i] == "졸음3"):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 쿨쿨 잠들어 있다.")
                if (random.randint(1, 100) <= 66):
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 잠에서 깨어났다!")
                    self.state[i] = "정상"
                else:
                    self.state[i] = "졸음3"
            elif (self.state[i] == "졸음4"):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 쿨쿨 잠들어 있다.")
                    print(f"{self.name}은(는) 잠에서 깨어났다!")
                self.state[i] = "정상"

            #얼음 코드
            if (self.state[i] == "얼음"):
                if (random.randint(1, 100) <= 20):
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 얼음 상태에서 벗어났다!")
                    self.state[i] = "정상"
                else:
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 얼어버려서 움직일 수 없다!")

            #화상 코드
            if (self.state[i] == "화상"):
                if (random.randint(1, 100) <= 20):
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 화상에서 나았다!")
                    self.attack = math.floor(self.attack * 2)
                    self.state[i] = "정상"
                else:
                    self.health = math.floor(self.health - self.health / 16)
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 화상 데미지를 입었다! 현재 체력: {self.health if self.health >= 0 else 0}/{self.maxhealth}")

            #마비 코드
            if (self.state[i] == "마비"):
                if (random.randint(1, 100) <= 25):
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 마비에서 회복되었다!")
                    self.attack = math.floor(self.speed * 2)
                    self.state[i] = "정상"
                elif (random.randint(1, 100) <= 30):
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 몸이 저려서 움직일 수 없다!")
                    self.state[i] = "전신마비"

            #중독 코드
            if (self.state[i] == "중독"):
                if (random.randint(1, 100) <= 15):
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 독을 이겨냈다!")
                    self.state[i] = "정상"
                else:
                    self.health = math.floor(self.health - self.health / 8)
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 독에 의한 데미지를 입었다! 현재 체력: {self.health if self.health >= 0 else 0}/{self.maxhealth}")

            #풀죽음
            if (self.state[i] == "풀죽음1"):
                if (not lists_have_common_values(self.state, ["자해", "졸음1", "졸음2", "졸음3", "얼음", "전신마비"])):
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 풀이 죽어 기술을 쓸 수 없다!")
                self.state[i] = "풀죽음2"
            elif (self.state[i] == "풀죽음2"):
                self.state[i] = "정상"

            #반동
            if (self.state[i] == "반동1"):
                if (not lists_have_common_values(self.state, ["자해", "졸음1", "졸음2", "졸음3", "얼음", "전신마비"])):
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 반동으로 움직일 수 없다!")
                self.state[i] = "반동2"
            elif (self.state[i] == "반동2"):
                self.state[i] = "정상"

            #난동 코드
            if (self.state[i] == "난동1"):
                self.state[i] = "난동2"
            elif (self.state[i] == "난동2"):
                if (random.randint(1, 100) <= 50):
                    self.state[i] = "난동3"
                else:
                    if ("혼란" not in self.state):
                        if (PRINT_FLAG):
                            print(f"{self.name}은(는) 혼란에 빠졌다!")
                        self.state[i] = "혼란"
                        random.randint(0, 3)
                    else:
                        self.state[i] = "정상"
            elif (self.state[i] == "난동3"):
                if ("혼란" not in self.state):
                        if (PRINT_FLAG):
                            print(f"{self.name}은(는) 혼란에 빠졌다!")
                        self.state[i] = "혼란"
                else:
                    self.state[i] = "정상"

            #혼란 코드
            if (self.state[i] == "혼란"):
                if (PRINT_FLAG):
                    print(f"{self.name}은(는) 혼란에 빠져있다!")
                if (random.randint(1, 100) <= 30):
                    if (PRINT_FLAG):
                        print(f"{self.name}은(는) 혼란이 풀렸다!")
                    self.state[i] = "정상"
                elif (random.randint(1, 100) <= 60):
                    self.health -= math.floor((self.attack * 40 / (self.maxhealth * self.defense / 0.611) * 85) / 3)
                    if (PRINT_FLAG):
                        print(f"영문도 모른 채 자신을 공격했다! 현재 체력: {self.health if self.health >= 0 else 0}/{self.maxhealth}")
                    self.state[i] = "자해"

        #제거 코드
        while "정상" in self.state:
            self.state.remove("정상")

    def compatibility_check(self, attack_type, type_list):
        compatibility = 1
        for defend_type in type_list:
            compatibility *= Enum.COMPATIBILITY_TABLE.value[Enum.TYPE_DICT.value[attack_type]][Enum.TYPE_DICT.value[defend_type]]
        return compatibility


    def type_mapping(self, type):
        return Enum.TYPE_DICT.value[type]
    def value_mapping(self, index):
        return list(Enum.TYPE_DICT.value.keys())[list(Enum.TYPE_DICT.value.values()).index(index)]

In [ ]:
#기술 리스트
skill1 = PokemonSkill("지진", 100, 100, "땅", 10, "물리")
skill2 = PokemonSkill("불대문자", 110, 85, "불꽃", 5, "특수")
skill3 = PokemonSkill("더스트슈트", 120, 80, "독", 5, "물리")
skill4 = PokemonSkill("번개", 110, 70, "전기", 10, "특수")
skill5 = PokemonSkill("리프블레이드", 90, 100, "풀", 15, "물리")
skill5 = PokemonSkill("볼부비부비", 20, 100, "전기", 20, "물리")
skill7 = PokemonSkill("사이코키네시스", 90, 100, "에스퍼", 10, "특수")
skill8 = PokemonSkill("볼트태클", 120, 100, "전기", 15, "물리")
skill9 = PokemonSkill("애교부리기", 0, 100, "노말", 20, "변화")
skill10 = PokemonSkill("독찌르기", 80, 100, "독", 20, "물리")
skill11 = PokemonSkill("불꽃펀치", 75, 100, "불꽃", 15, "물리")
skill12 = PokemonSkill("냉동펀치", 75, 100, "얼음", 15, "물리")
skill13 = PokemonSkill("사이코커터", 70, 100, "에스퍼", 20, "물리")
skill14 = PokemonSkill("박치기", 70, 100, "독", 15, "물리")
skill15 = PokemonSkill("번개펀치", 75, 100, "전기", 15, "물리")
skill16 = PokemonSkill("시그널빔", 75, 100, "벌레", 15, "특수")
skill17 = PokemonSkill("크로스촙", 100, 80, "격투", 5, "물리")
skill18 = PokemonSkill("얼음펀치", 75, 100, "얼음", 15, "물리")
skill19 = PokemonSkill("눈보라", 110, 70, "얼음", 5, "특수")
skill20 = PokemonSkill("드래곤클로", 80, 100, "드래곤", 15, "물리")
skill21 = PokemonSkill("바위깨기", 40, 100, "격투", 15, "물리")
skill22 = PokemonSkill("솔라빔", 120, 100, "풀", 10, "특수")
skill23 = PokemonSkill("파도타기", 90, 100, "물", 15, "특수")
skill24 = PokemonSkill("방전", 80, 100, "전기", 15, "특수")
skill25 = PokemonSkill("폭포오르기", 80, 90, "물", 15, "물리")
skill26 = PokemonSkill("웨이브태클", 120, 100, "물", 10, "물리")
skill27 = PokemonSkill("쾌청", 0, 100, "불꽃", 5, "변화")
skill28 = PokemonSkill("매지컬리프", 60, 100, "풀", 20, "특수")
skill29 = PokemonSkill("깜짝베기", 70, 100, "악", 15, "물리")
skill30 = PokemonSkill("전광석화", 40, 100, "노말", 30, "물리")
skill31 = PokemonSkill("파괴광선", 150, 90, "노말", 5, "특수")
skill32 = PokemonSkill("탁류", 90, 85, "물", 10, "특수")
skill33 = PokemonSkill("회전부리", 80, 100, "비행", 20, "물리")
skill34 = PokemonSkill("화염방사", 90, 100, "불꽃", 15, "특수")
skill35 = PokemonSkill("대지의힘", 90, 100, "땅", 10, "특수")
skill36 = PokemonSkill("에어슬래시", 75, 95, "비행", 15, "물리")
skill37 = PokemonSkill("냉동빔", 90, 100, "얼음", 10, "특수")
skill38 = PokemonSkill("더블어택", 70, 90, "노말", 10, "물리")
skill39 = PokemonSkill("섀도클로", 70, 100, "고스트", 15, "물리")
skill40 = PokemonSkill("락블레스트", 100, 80, "바위", 10, "물리")
skill41 = PokemonSkill("힘껏치기", 100, 85, "노말", 15, "물리")
skill42 = PokemonSkill("괴상한바람", 60, 100, "고스트", 5, "특수")
skill43 = PokemonSkill("무릎차기", 130, 70, "격투", 10, "물리")
skill44 = PokemonSkill("마하펀치", 40, 100, "격투", 30, "물리")
skill45 = PokemonSkill("깨트리기", 75, 100, "격투", 15, "물리")
skill46 = PokemonSkill("전자포", 80, 100, "전기", 15, "특수")
skill47 = PokemonSkill("얼어붙은바람", 65, 95, "얼음", 20, "특수")
skill48 = PokemonSkill("속이기", 40, 80, "노말", 10, "물리")
skill49 = PokemonSkill("에너지볼", 90, 100, "풀", 10, "특수")
skill50 = PokemonSkill("용의파동", 85, 100, "드래곤", 10, "특수")
skill51 = PokemonSkill("얼음엄니", 65, 95, "얼음", 15, "물리")
skill52 = PokemonSkill("하품", 0, 100, "악", 20, "변화")
skill53 = PokemonSkill("게으름피우기", 0, 100, "노말", 5, "변화")
skill54 = PokemonSkill("신속", 80, 100, "노말", 5, "물리")
skill55 = PokemonSkill("칼춤", 0, 100, "노말", 20, "변화")
skill56 = PokemonSkill("인파이트", 120, 100, "격투", 5, "물리")
skill57 = PokemonSkill("불릿펀치", 75, 100, "강철", 15, "물리")
skill58 = PokemonSkill("파멸의소용돌이", 120, 85, "악", 5, "특수")
skill59 = PokemonSkill("스파크", 65, 100, "전기", 20, "물리")
skill60 = PokemonSkill("에어컷터", 60, 95, "비행", 25, "특수")
skill61 = PokemonSkill("암해머", 100, 90, "격투", 10, "물리")
skill62 = PokemonSkill("아쿠아브레이크", 85, 100, "물", 10, "물리")
skill63 = PokemonSkill("플레어드라이브", 120, 100, "불꽃", 15, "물리")
skill64 = PokemonSkill("맹독", 0, 90, "독", 10, "변화")
skill65 = PokemonSkill("자이로볼", 0, 100, "강철", 5, "변화")
skill66 = PokemonSkill("강철날개", 70, 90, "강철", 25, "물리")
skill67 = PokemonSkill("눈사태", 120, 90, "얼음", 5, "특수")
skill68 = PokemonSkill("우드해머", 120, 100, "풀", 15, "물리")
skill69 = PokemonSkill("깨물어부수기", 80, 100, "악", 15, "물리")
skill70 = PokemonSkill("스톤에지", 100, 80, "바위", 5, "물리")
skill71 = PokemonSkill("스톤샤워", 75, 90, "바위", 15, "물리")
skill72 = PokemonSkill("하이드로펌프", 110, 80, "물", 5, "특수")
skill73 = PokemonSkill("러스터캐논", 80, 100, "강철", 10, "특수")
skill74 = PokemonSkill("아쿠아제트", 40, 100, "물", 20, "물리")
skill75 = PokemonSkill("이판사판태클", 120, 100, "노말", 15, "물리")
skill76 = PokemonSkill("브레이브버드", 120, 100, "노말", 15, "물리")
skill77 = PokemonSkill("록클라임", 90, 85, "노말", 20, "물리")
skill78 = PokemonSkill("벌레의야단법석", 90, 100, "벌레", 10, "특수")
skill79 = PokemonSkill("제비반환", 60, 100, "비행", 20, "물리")
skill80 = PokemonSkill("하이퍼보이스", 90, 100, "노말", 10, "특수")
skill81 = PokemonSkill("벌레먹기", 60, 100, "벌레", 20, "물리")
skill82 = PokemonSkill("10만볼트", 90, 100, "전기", 15, "특수")
skill83 = PokemonSkill("번개엄니", 65, 95, "전기", 15, "물리")
skill84 = PokemonSkill("아이언테일", 100, 75, "강철", 15, "물리")
skill85 = PokemonSkill("꽃잎댄스", 120, 100, "풀", 10, "특수")
skill86 = PokemonSkill("오물폭탄", 90, 100, "독", 10, "특수")
skill87 = PokemonSkill("섀도볼", 80, 100, "고스트", 15, "특수")
skill88 = PokemonSkill("기가드레인", 75, 100, "풀", 10, "특수")
skill89 = PokemonSkill("사념의박치기", 80, 90, "에스퍼", 15, "물리")
skill90 = PokemonSkill("양날박치기", 150, 80, "바위", 5, "물리")
skill91 = PokemonSkill("아이언헤드", 80, 100, "강철", 15, "물리")
skill92 = PokemonSkill("나비춤", 0, 100, "벌레", 20, "변화")
skill93 = PokemonSkill("은빛바람", 60, 100, "벌레", 5, "특수")
skill94 = PokemonSkill("공격지령", 90, 100, "벌레", 15, "물리")
skill95 = PokemonSkill("방어지령", 0, 100, "벌레", 10, "변화")
skill96 = PokemonSkill("회복지령", 0, 100, "벌레", 10, "변화")
skill97 = PokemonSkill("파워젬", 80, 100, "바위", 20, "특수")
skill98 = PokemonSkill("분노의앞니", 0, 90, "노말", 10, "물리")
skill99 = PokemonSkill("최면술", 0, 60, "에스퍼", 20, "변화")
skill100 = PokemonSkill("기습", 70, 100, "악", 5, "물리")
skill101 = PokemonSkill("치료방울", 0, 100, "노말", 5, "변화")
skill102 = PokemonSkill("보복", 0, 100, "악", 10, "변화")
skill103 = PokemonSkill("안다리걸기", 50, 100, "격투", 20, "물리")
skill104 = PokemonSkill("기합구슬", 120, 70, "격투", 5, "특수")
skill105 = PokemonSkill("알낳기", 0, 100, "노말", 5, "변화")
skill106 = PokemonSkill("공중날기", 90, 95, "비행", 15, "물리")
skill107 = PokemonSkill("수다", 40, 80, "비행", 20, "특수")
skill108 = PokemonSkill("악의파동", 80, 100, "악", 15, "특수")
skill109 = PokemonSkill("용성군", 130, 90, "드래곤", 5, "특수")
skill110 = PokemonSkill("누르기", 85, 100, "노말", 15, "물리")
skill111 = PokemonSkill("씨폭탄", 80, 100, "풀", 15, "물리")
skill112 = PokemonSkill("크로스포이즌", 70, 100, "독", 20, "물리")
skill113 = PokemonSkill("드레인펀치", 75, 100, "격투", 10, "물리")
skill114 = PokemonSkill("시저크로스", 80, 100, "벌레", 15, "물리")
skill115 = PokemonSkill("파워휩", 120, 85, "풀", 10, "물리")
skill116 = PokemonSkill("비축하기", 0, 100, "노말", 20, "변화")
skill117 = PokemonSkill("열탕", 80, 100, "물", 15, "특수")
skill118 = PokemonSkill("비바라기", 0, 100, "물", 5, "변화")
skill119 = PokemonSkill("이상한빛", 0, 85, "고스트", 10, "변화")
skill120 = PokemonSkill("리프스톰", 130, 90, "풀", 5, "특수")
skill121 = PokemonSkill("핥기", 40, 100, "고스트", 30, "물리")
skill122 = PokemonSkill("메가혼", 120, 85, "벌레", 10, "물리")
skill123 = PokemonSkill("역린", 120, 100, "드래곤", 10, "물리")
skill124 = PokemonSkill("원시의힘", 60, 100, "바위", 5, "특수")
skill125 = PokemonSkill("파동탄", 85, 100, "격투", 20, "특수")
skill126 = PokemonSkill("물의파동", 60, 100, "물", 20, "특수")
skill127 = PokemonSkill("불새", 140, 90, "비행", 5, "물리")
skill128 = PokemonSkill("얼음뭉치", 40, 100, "얼음", 30, "물리")
skill129 = PokemonSkill("엄청난힘", 120, 100, "격투", 5, "물리")
skill130 = PokemonSkill("난동부리기", 120, 100, "노말", 10, "물리")
skill131 = PokemonSkill("트라이어택", 80, 100, "노말", 10, "특수")
skill132 = PokemonSkill("야습", 40, 100, "고스트", 30, "물리")
skill133 = PokemonSkill("암석봉인", 60, 95, "바위", 15, "물리")
skill134 = PokemonSkill("블레이즈킥", 85, 90, "불꽃", 10, "물리")
skill135 = PokemonSkill("괴력", 80, 100, "노말", 15, "물리")
skill136 = PokemonSkill("은혜갚기", 90, 100, "노말", 10, "물리")
skill137 = PokemonSkill("배북", 0, 100, "노말", 10, "변화")
skill138 = PokemonSkill("벌레의저항", 50, 100, "벌레", 20, "특수")
skill139 = PokemonSkill("폭풍", 110, 70, "비행", 10, "특수")
skill140 = PokemonSkill("버섯포자", 0, 100, "풀", 10, "변화")
skill141 = PokemonSkill("스카이어퍼", 85, 90, "격투", 15, "물리")
skill142 = PokemonSkill("속여때리기", 60, 100, "악", 20, "물리")
skill143 = PokemonSkill("메탈클로", 50, 95, "강철", 35, "물리")
skill144 = PokemonSkill("폭음파", 140, 100, "노말", 10, "특수")
skill145 = PokemonSkill("오버히트", 130, 90, "불꽃", 5, "특수")
skill146 = PokemonSkill("손바닥치기", 15, 100, "격투", 20, "물리")
skill147 = PokemonSkill("나쁜음모", 0, 100, "악", 20, "변화")
skill148 = PokemonSkill("전기자석파", 0, 90, "전기", 20, "변화")
skill149 = PokemonSkill("진흙폭탄", 65, 85, "땅", 10, "특수")
skill150 = PokemonSkill("도깨비불", 0, 85, "불꽃", 15, "변화")
skill151 = PokemonSkill("불꽃엄니", 65, 95, "불꽃", 15, "물리")
skill152 = PokemonSkill("플래시", 0, 100, "노말", 20, "변화")
skill153 = PokemonSkill("고속이동", 0, 100, "에스퍼", 30, "변화")
skill154 = PokemonSkill("꿀꺽", 0, 100, "노말", 20, "변화")
skill155 = PokemonSkill("헤수스파우팅", 150, 100, "물", 5, "특수")
skill156 = PokemonSkill("열풍", 95, 90, "불꽃", 10, "특수")
skill157 = PokemonSkill("분연", 80, 100, "불꽃", 15, "특수")
skill158 = PokemonSkill("철벽", 0, 100, "강철", 15, "변화")
skill159 = PokemonSkill("망각술", 0, 100, "에스퍼", 15, "변화")
skill160 = PokemonSkill("흔들흔들댄스", 0, 100, "노말", 10, "변화")
skill161 = PokemonSkill("브레이크클로", 75, 95, "노말", 10, "물리")
skill162 = PokemonSkill("포이즌테일", 50, 100, "독", 25, "물리")
skill163 = PokemonSkill("뱀눈초리", 0, 100, "노말", 30, "변화")
skill164 = PokemonSkill("아쿠아테일", 90, 90, "물", 10, "물리")
skill165 = PokemonSkill("진흙뿌리기", 20, 100,"땅", 10, "특수")
skill166 = PokemonSkill("환상빔", 65, 100, "에스퍼", 20, "특수")
skill167 = PokemonSkill("오물웨이브", 95, 100, "독", 10, "특수")
skill168 = PokemonSkill("록커트", 0, 80, "바위", 20, "변화")
skill169 = PokemonSkill("록블라스트", 25, 90, "바위", 10, "물리")
skill170 = PokemonSkill("HP회복", 0, 100, "노말", 5, "변화")
skill171 = PokemonSkill("웨더볼", 50, 100, "노말", 10, "특수")
skill172 = PokemonSkill("속임수", 95, 100, "악", 15, "물리")
skill173 = PokemonSkill("안개제거", 0, 100, "비행", 15, "변화")
skill174 = PokemonSkill("풀베기", 50, 95, "노말", 30, "물리")
skill175 = PokemonSkill("멸망의노래", 0, 90, "노말", 5, "변화")
skill176 = PokemonSkill("신비의부적", 0, 80, "노말", 25, "변화")
skill177 = PokemonSkill("카운터", 0, 100, "격투", 20, "물리")
skill178 = PokemonSkill("미러코트", 0, 100, "에스퍼", 20, "특수")
skill179 = PokemonSkill("싸라기눈", 0, 100, "얼음", 5, "변화")
skill180 = PokemonSkill("절대영도", 0, 30, "얼음", 5, "특수")
skill181 = PokemonSkill("뛰어오르기", 85, 95, "비행", 5, "물리")
skill182 = PokemonSkill("천사의키스", 0, 75, "노말", 10, "변화")
skill183 = PokemonSkill("용의춤", 0, 100, "드래곤", 20, "변화")
skill184 = PokemonSkill("코멧펀치", 90, 90, "강철", 10, "물리")
skill185 = PokemonSkill("불사르기", 130, 100, "불꽃", 5, "특수")
skill186 = PokemonSkill("분화", 150, 75, "불꽃", 5, "특수")
skill187 = PokemonSkill("정리정돈", 0, 100, "노말", 10, "변화")
skill188 = PokemonSkill("구멍파기", 80, 100, "땅", 10, "물리")
skill189 = PokemonSkill("노래하기", 0, 55, "노말", 15, "변화")
skill190 = PokemonSkill("수면가루", 0, 75, "풀", 15, "변화")
skill191 = PokemonSkill("순풍", 0, 100, "비행", 15, "변화")
skill192 = PokemonSkill("저리가루", 0, 100, "풀", 30, "변화")
skill193 = PokemonSkill("아침햇살", 0, 100, "노말", 5, "변화")
skill194 = PokemonSkill("달빛", 0, 100, "노말", 5, "변화")
skill195 = PokemonSkill("바크아웃", 55, 95, "악", 15, "특수")
skill196 = PokemonSkill("잠재파워", 60, 100, "노말", 15, "특수")
skill197 = PokemonSkill("트윈빔", 80, 100, "에스퍼", 15, "특수")
skill198 = PokemonSkill("따라가때리기", 40, 100, "악", 20, "물리")
skill199 = PokemonSkill("독침천발", 80, 100, "독", 15, "물리")
skill200 = PokemonSkill("작아지기", 0, 100, "노말", 10, "변화")
skill201 = PokemonSkill("날개치기", 60, 100, "비행", 35, "물리")
skill202 = PokemonSkill("숲의저주", 0, 100, "풀", 20, "변화")
skill203 = PokemonSkill("파워트릭", 0, 100, "에스퍼", 10, "변화")
skill204 = PokemonSkill("객기", 70, 100, "노말", 20, "물리")
skill205 = PokemonSkill("벌크업", 0, 100, "격투", 20, "변화")
skill206 = PokemonSkill("대포무노포", 65, 85, "물", 10, "특수")
skill207 = PokemonSkill("프리젠트", 50, 90, "노말", 15, "물리")
skill208 = PokemonSkill("배리어러시", 70, 90, "에스퍼", 15, "물리")
skill209 = PokemonSkill("짓밟기", 65, 100, "노말", 20, "물리")
skill210 = PokemonSkill("와일드볼트", 90, 100, "전기", 15, "물리")
skill211 = PokemonSkill("트리플킥", 60, 90, "격투", 10, "물리")
skill212 = PokemonSkill("악마의키스", 0, 75, "노말", 10, "변화")
skill213 = PokemonSkill("우유마시기", 0, 100, "노말", 5, "변화")
skill214 = PokemonSkill("사이코쇼크", 80, 100, "에스퍼", 10, "특수")
skill215 = PokemonSkill("꽃보라", 90, 100, "풀", 15, "물리")
skill216 = PokemonSkill("스피드스타", 60, 100, "노말", 20, "특수")
skill217 = PokemonSkill("숟가락휘기", 0, 80, "에스퍼", 15, "변화")
skill218 = PokemonSkill("폭발펀치", 100, 50, "격투", 5, "물리")
skill219 = PokemonSkill("집게해머", 100, 90, "물", 10, "물리")
skill220 = PokemonSkill("싫은소리", 0, 85, "노말", 20, "변화")
skill221 = PokemonSkill("잼잼펀치", 70, 100, "노말", 10, "물리")
skill222 = PokemonSkill("뿔드릴", 0, 30, "노말", 10, "물리")
skill223 = PokemonSkill("변신", 0, 100, "노말", 10, "변화")
skill224 = PokemonSkill("녹기", 0, 100, "독", 20, "변화")
skill225 = PokemonSkill("기가임팩트", 150, 90, "노말", 5, "물리")
skill226 = PokemonSkill("고드름떨구기", 85, 90, "얼음", 10, "물리")
skill227 = PokemonSkill("철제광선", 140, 95, "강철", 5, "특수")
skill228 = PokemonSkill("발버둥", 50, 100, "", 999, "물리")
#포켓몬 리스트
pokemon1 = Pokemon("루카리오", ["격투", "강철"], -1, -1, 70, 145, 88, 140, 70, 112, [copy.deepcopy(skill54), copy.deepcopy(skill55), copy.deepcopy(skill56), copy.deepcopy(skill57)]) #신속, 칼춤, 인파이트, 불릿펀치
pokemon2 = Pokemon("하마돈", ["땅"], -1, -1, 108, 112, 118, 68, 72, 47, [copy.deepcopy(skill1), copy.deepcopy(skill51), copy.deepcopy(skill52), copy.deepcopy(skill53)]) #지진, 얼음엄니, 하품, 게으름피우기
pokemon3 = Pokemon("토대부기", ["풀", "땅"], -1, -1, 95, 109, 105, 75, 85, 56, [copy.deepcopy(skill1), copy.deepcopy(skill68), copy.deepcopy(skill129), copy.deepcopy(skill70)]) #지진, 우드해머, 엄청난힘, 스톤에지
pokemon4 = Pokemon("초염몽", ["불꽃", "격투"], -1, -1, 76, 104, 71, 104, 71, 108, [copy.deepcopy(skill63), copy.deepcopy(skill44), copy.deepcopy(skill15), copy.deepcopy(skill56)]) #플레어드라이브, 마하펀치, 번개펀치, 인파이트
pokemon5 = Pokemon("엠페르트", ["물", "강철"], -1, -1, 84, 86, 88, 111, 101, 60, [copy.deepcopy(skill74), copy.deepcopy(skill72), copy.deepcopy(skill73), copy.deepcopy(skill37)]) #아쿠아제트, 하이드로펌프, 러스터캐논, 냉동빔
pokemon6 = Pokemon("찌르호크", ["노말", "비행"], -1, -1, 85, 120, 70, 50, 60, 100, [copy.deepcopy(skill75), copy.deepcopy(skill76), copy.deepcopy(skill56), copy.deepcopy(skill30)]) #이판사판태클, 브레이브버드, 인파이트, 전광석화
pokemon7 = Pokemon("비버통", ["노말", "물"], -1, -1, 79, 85, 60, 55, 60, 71, [copy.deepcopy(skill75), copy.deepcopy(skill21), copy.deepcopy(skill23), copy.deepcopy(skill77)]) #이판사판태클, 바위깨기, 파도타기, 록클라임
pokemon8 = Pokemon("귀뚤톡크", ["벌레"], -1, -1, 77, 85, 51, 55, 51, 65, [copy.deepcopy(skill78), copy.deepcopy(skill79), copy.deepcopy(skill80), copy.deepcopy(skill81)]) #벌레의야단법석, 제비반환, 하이퍼보이스, 벌레먹기
pokemon9 = Pokemon("렌트라", ["전기"], -1, -1, 80, 120, 79, 95, 79, 70, [copy.deepcopy(skill51), copy.deepcopy(skill210), copy.deepcopy(skill69), copy.deepcopy(skill84)]) #얼음엄니, 와일드볼트, 깨물어부수기, 아이언테일
pokemon10 = Pokemon("로즈레이드", ["풀", "독"], -1, -1, 60, 70, 65, 125, 105, 90, [copy.deepcopy(skill120), copy.deepcopy(skill86), copy.deepcopy(skill87), copy.deepcopy(skill190)]) #리프스톰, 오물폭탄, 섀도볼, 기가드레인
pokemon11 = Pokemon("램펄드", ["바위"], -1, -1, 97, 165, 60, 65, 50, 58, [copy.deepcopy(skill70), copy.deepcopy(skill89), copy.deepcopy(skill90), copy.deepcopy(skill11)]) #스톤에지, 사념의박치기, 양날박치기, 불꽃펀치
pokemon12 = Pokemon("바리톱스", ["바위", "강철"], -1, -1, 60, 52, 168, 47, 138, 30, [copy.deepcopy(skill71), copy.deepcopy(skill73), copy.deepcopy(skill91), copy.deepcopy(skill1)]) #스톤샤워, 러스터캐논, 아이언헤드, 지진
pokemon13 = Pokemon("나메일", ["벌레", "비행"], -1, -1, 70, 94, 50, 94, 50, 66, [copy.deepcopy(skill92), copy.deepcopy(skill7), copy.deepcopy(skill36), copy.deepcopy(skill93)]) #나비춤, 사이코키네시스, 에어슬래시, 은빛바람
pokemon14 = Pokemon("비퀸", ["벌레", "비행"], -1, -1, 70, 80, 102, 80, 102, 40, [copy.deepcopy(skill94), copy.deepcopy(skill95), copy.deepcopy(skill96), copy.deepcopy(skill97)]) #공격지령, 방어지령, 회복지령, 파워젬
pokemon15 = Pokemon("파치리스", ["전기"], -1, -1, 60, 45, 70, 45, 90, 95, [copy.deepcopy(skill98), copy.deepcopy(skill5), copy.deepcopy(skill9), copy.deepcopy(skill24)]) #분노의앞니, 볼부비부비, 애교부리기, 방전
pokemon16 = Pokemon("플로젤", ["물"], -1, -1, 85, 105, 55, 85, 50, 115, [copy.deepcopy(skill12), copy.deepcopy(skill69), copy.deepcopy(skill45), copy.deepcopy(skill26)]) #냉동펀치, 깨물어부수기, 깨트리기, 웨이브태클
pokemon17 = Pokemon("체리꼬", ["풀"], -1, -1, 70, 90, 70, 87, 117, 85, [copy.deepcopy(skill27), copy.deepcopy(skill28), copy.deepcopy(skill85), copy.deepcopy(skill22)]) #쾌청, 매지컬리프, 꽃잎댄스, 솔라빔
pokemon18 = Pokemon("트리토돈", ["물", "땅"], -1, -1, 111, 83, 68, 92, 82, 39, [copy.deepcopy(skill117), copy.deepcopy(skill35), copy.deepcopy(skill37), copy.deepcopy(skill170)]) #열탕, 대지의힘, 냉동빔, HP회복
pokemon19 = Pokemon("겟헨보숭", ["노말"], -1, -1, 75, 100, 66, 60, 66, 115, [copy.deepcopy(skill38), copy.deepcopy(skill39), copy.deepcopy(skill45), copy.deepcopy(skill84)]) #더블어택, 섀도클로, 깨트리기, 아이언테일
pokemon20 = Pokemon("둥실라이드", ["고스트", "비행"], -1, -1, 150, 80, 44, 90, 54, 80, [copy.deepcopy(skill87), copy.deepcopy(skill7), copy.deepcopy(skill82), copy.deepcopy(skill150)]) #섀도볼, 사이코키네시스, 10만볼트, 도깨비불
pokemon21 = Pokemon("이어롭", ["노말"], -1, -1, 65, 136, 94, 54, 96, 135, [copy.deepcopy(skill43), copy.deepcopy(skill11), copy.deepcopy(skill84), copy.deepcopy(skill48)]) #무릎차기, 불꽃펀치, 아이언테일, 속이기
pokemon22 = Pokemon("무우마직", ["고스트"], -1, -1, 60, 60, 60, 105, 105, 105, [copy.deepcopy(skill87), copy.deepcopy(skill49), copy.deepcopy(skill97), copy.deepcopy(skill7)]) #섀도볼, 에너지볼, 파워젬, 사이코키네시스
pokemon23 = Pokemon("돈크로우", ["악", "비행"], -1, -1, 100, 125, 52, 105, 52, 71, [copy.deepcopy(skill29), copy.deepcopy(skill79), copy.deepcopy(skill66), copy.deepcopy(skill7)]) #깜짝베기, 제비반환, 강철날개, 사이코키네시스
pokemon24 = Pokemon("몬냥이", ["노말"], -1, -1, 71, 82, 64, 64, 59, 112, [copy.deepcopy(skill99), copy.deepcopy(skill48), copy.deepcopy(skill100), copy.deepcopy(skill98)]) #최면술, 속이기, 기습, 분노의앞니
pokemon25 = Pokemon("치렁", ["에스퍼"], -1, -1, 75, 50, 80, 95, 90, 65, [copy.deepcopy(skill101), copy.deepcopy(skill99), copy.deepcopy(skill7), copy.deepcopy(skill16)]) #치료방울, 최면술, 사이코키네시스, 시그널빔
pokemon26 = Pokemon("스컹탱크", ["독", "악"], -1, -1, 103, 93, 67, 71, 61, 84, [copy.deepcopy(skill10), copy.deepcopy(skill3), copy.deepcopy(skill29), copy.deepcopy(skill2)]) #독찌르기, 더스트슈트, 깜짝베기, 불대문자
pokemon27 = Pokemon("동탁군", ["강철", "에스퍼"], -1, -1, 67, 89, 116, 79, 116, 33, [copy.deepcopy(skill65), copy.deepcopy(skill89), copy.deepcopy(skill1), copy.deepcopy(skill71)]) #자이로볼, 사념의박치기, 지진, 스톤샤워
pokemon28 = Pokemon("꼬지모", ["바위"], -1, -1, 70, 100, 115, 30, 65, 30, [copy.deepcopy(skill68), copy.deepcopy(skill90), copy.deepcopy(skill103), copy.deepcopy(skill61)]) #우드해머, 양날박치기, 안다리걸기, 암해머
pokemon29 = Pokemon("마임맨", ["에스퍼"], -1, -1, 40, 45, 65, 100, 120, 90, [copy.deepcopy(skill7), copy.deepcopy(skill49), copy.deepcopy(skill82), copy.deepcopy(skill104)]) #사이코키네시스, 에너지볼, 10만볼트, 기합구슬
pokemon30 = Pokemon("해피너스", ["노말"], -1, -1, 255, 10, 10, 75, 135, 55, [copy.deepcopy(skill105), copy.deepcopy(skill82), copy.deepcopy(skill37), copy.deepcopy(skill34)]) #알낳기, 10만볼트, 냉동빔, 화염방사
pokemon31 = Pokemon("페라페", ["노말", "비행"], -1, -1, 76, 65, 45, 92, 42, 91, [copy.deepcopy(skill80), copy.deepcopy(skill60), copy.deepcopy(skill106), copy.deepcopy(skill107)]) #하이퍼보이스, 에어컷터, 공중날기, 수다
pokemon32 = Pokemon("화강돌", ["고스트", "악"], -1, -1, 50, 92, 108, 92, 108, 35, [copy.deepcopy(skill100), copy.deepcopy(skill132), copy.deepcopy(skill150), copy.deepcopy(skill198)]) #기습, 야습, 도깨비불, 따라가때리기
pokemon33 = Pokemon("한카리아스", ["드래곤", "땅"], -1, -1, 108, 170, 115, 120, 95, 92, [copy.deepcopy(skill1), copy.deepcopy(skill91), copy.deepcopy(skill55), copy.deepcopy(skill123)]) #지진, 아이언헤드, 칼춤, 역린
pokemon34 = Pokemon("잠만보", ["노말"], -1, -1, 160, 110, 65, 65, 110, 30, [copy.deepcopy(skill69), copy.deepcopy(skill110), copy.deepcopy(skill91), copy.deepcopy(skill111)]) #깨물어부수기, 누르기, 아이언헤드, 씨폭탄
pokemon35 = Pokemon("드래피온", ["독", "악"], -1, -1, 70, 90, 110, 60, 75, 95, [copy.deepcopy(skill69), copy.deepcopy(skill29), copy.deepcopy(skill3), copy.deepcopy(skill17)]) #깨물어부수기, 깜짝베기, 더스트슈트, 크로스촙
pokemon36 = Pokemon("독개굴", ["독", "격투"], -1, -1, 83, 106, 65, 86, 65, 85, [copy.deepcopy(skill10), copy.deepcopy(skill113), copy.deepcopy(skill114), copy.deepcopy(skill1)]) #독찌르기, 드레인펀치, 시저크로스, 지진
pokemon37 = Pokemon("무스틈니", ["풀"], -1, -1, 74, 100, 72, 90, 72, 46, [copy.deepcopy(skill115), copy.deepcopy(skill116), copy.deepcopy(skill69), copy.deepcopy(skill86)]) #파워휩, 비축하기, 깨물어부수기, 오물폭탄
pokemon38 = Pokemon("네오라이트", ["물"], -1, -1, 69, 69, 76, 69, 86, 91, [copy.deepcopy(skill23), copy.deepcopy(skill117), copy.deepcopy(skill118), copy.deepcopy(skill119)]) #파도타기, 열탕, 비바라기, 이상한빛
pokemon39 = Pokemon("만타인", ["물", "비행"], -1, -1, 85, 40, 70, 80, 140, 70, [copy.deepcopy(skill23), copy.deepcopy(skill36), copy.deepcopy(skill37), copy.deepcopy(skill16)]) #파도타기, 에어슬래시, 냉동빔, 시그널빔
pokemon40 = Pokemon("눈설왕", ["풀", "얼음"], -1, -1, 90, 132, 105, 132, 105, 30, [copy.deepcopy(skill68), copy.deepcopy(skill71), copy.deepcopy(skill19), copy.deepcopy(skill35)]) #우드해머, 스톤샤워, 눈보라, 대지의힘
pokemon41 = Pokemon("포푸니라", ["악", "얼음"], -1, -1, 70, 120, 65, 45, 85, 125, [copy.deepcopy(skill29), copy.deepcopy(skill12), copy.deepcopy(skill45), copy.deepcopy(skill226)]) #깜짝베기, 냉동펀치, 깨트리기, 고드름떨구기
pokemon42 = Pokemon("자포코일", ["전기", "강철"], -1, -1, 70, 70, 115, 130, 90, 60, [copy.deepcopy(skill82), copy.deepcopy(skill73), copy.deepcopy(skill227), copy.deepcopy(skill4)]) #10만볼트, 러스터캐논, 철제광선, 번개
pokemon43 = Pokemon("내룸벨트", ["노말"], -1, -1, 110, 85, 95, 80, 95, 50, [copy.deepcopy(skill115), copy.deepcopy(skill121), copy.deepcopy(skill55), copy.deepcopy(skill31)]) #파워휩, 핥기, 칼춤, 파괴광선
pokemon44 = Pokemon("거대코뿌리", ["땅", "바위"], -1, -1, 115, 140, 130, 55, 55, 40, [copy.deepcopy(skill1), copy.deepcopy(skill71), copy.deepcopy(skill122), copy.deepcopy(skill129)]) #지진, 스톤샤워, 메가혼, 엄청난힘
pokemon45 = Pokemon("덩쿠림보", ["풀"], -1, -1, 100, 100, 125, 110, 50, 50, [copy.deepcopy(skill115), copy.deepcopy(skill1), copy.deepcopy(skill124), copy.deepcopy(skill86)]) #파워휩, 지진, 원시의힘, 오물폭탄
pokemon46 = Pokemon("에레키블", ["전기"], -1, -1, 75, 123, 67, 95, 85, 95, [copy.deepcopy(skill210), copy.deepcopy(skill1), copy.deepcopy(skill12), copy.deepcopy(skill71)]) #와일드볼트, 지진, 냉동펀치, 스톤샤워
pokemon47 = Pokemon("마그마번", ["불꽃"], -1, -1, 75, 95, 67, 125, 95, 83, [copy.deepcopy(skill34), copy.deepcopy(skill82), copy.deepcopy(skill7), copy.deepcopy(skill104)]) #화염방사, 10만볼트, 사이코키네시스, 기합구슬
pokemon48 = Pokemon("토게키스", ["비행"], -1, -1, 85, 50, 95, 120, 115, 80, [copy.deepcopy(skill36), copy.deepcopy(skill125), copy.deepcopy(skill2), copy.deepcopy(skill87)]) #에어슬래시, 파동탄, 불대문자, 섀도볼
pokemon49 = Pokemon("메가자리", ["벌레", "비행"], -1, -1, 86, 76, 86, 116, 56, 95, [copy.deepcopy(skill36), copy.deepcopy(skill78), copy.deepcopy(skill124), copy.deepcopy(skill7)]) #에어슬래시, 벌레의야단법석, 원시의힘, 사이코키네시스
pokemon50 = Pokemon("리피아", ["풀"], -1, -1, 65, 110, 130, 60, 65, 95, [copy.deepcopy(skill5), copy.deepcopy(skill114), copy.deepcopy(skill84), copy.deepcopy(skill52)]) #리프블레이드, 시저크로스, 아이언테일, 하품
pokemon51 = Pokemon("글레이시아", ["풀"], -1, -1, 65, 60, 110, 130, 95, 65, [copy.deepcopy(skill37), copy.deepcopy(skill87), copy.deepcopy(skill126), copy.deepcopy(skill19)]) #냉동빔, 섀도볼, 물의파동, 눈보라
pokemon52 = Pokemon("글라이온", ["땅", "비행"], -1, -1, 75, 95, 125, 45, 75, 95, [copy.deepcopy(skill1), copy.deepcopy(skill79), copy.deepcopy(skill114), copy.deepcopy(skill127)]) #지진, 제비반환, 시저크로스, 눈보라
pokemon53 = Pokemon("맘모꾸리", ["얼음", "땅"], -1, -1, 110, 130, 80, 70, 60, 80, [copy.deepcopy(skill128), copy.deepcopy(skill1), copy.deepcopy(skill226), copy.deepcopy(skill70)]) #얼음뭉치, 지진, 고드름떨구기, 스톤에지
pokemon54 = Pokemon("폴리곤Z", ["노말"], -1, -1, 85, 80, 70, 135, 75, 90, [copy.deepcopy(skill131), copy.deepcopy(skill16), copy.deepcopy(skill108), copy.deepcopy(skill82)]) #트라이어택, 시그널빔, 악의파동, 10만볼트
pokemon55 = Pokemon("엘레이드", ["에스퍼", "격투"], -1, -1, 68, 165, 95, 65, 115, 110, [copy.deepcopy(skill89), copy.deepcopy(skill56), copy.deepcopy(skill132), copy.deepcopy(skill12)]) #사념의박치기, 인파이트, 야습, 냉동펀치
pokemon56 = Pokemon("대코파스", ["바위", "강철"], -1, -1, 60, 55, 145, 75, 150, 40, [copy.deepcopy(skill97), copy.deepcopy(skill73), copy.deepcopy(skill35), copy.deepcopy(skill82)]) #파워젬, 러스터캐논, 대지의힘, 10만볼트
pokemon57 = Pokemon("야느와르몽", ["고스트"], -1, -1, 45, 100, 135, 65, 135, 45, [copy.deepcopy(skill39), copy.deepcopy(skill132), copy.deepcopy(skill15), copy.deepcopy(skill1)]) #섀도클로, 야습, 번개펀치, 지진
pokemon58 = Pokemon("눈여아", ["얼음", "고스트"], -1, -1, 70, 80, 70, 80, 70, 110, [copy.deepcopy(skill37), copy.deepcopy(skill87), copy.deepcopy(skill82), copy.deepcopy(skill7)]) #냉동빔, 섀도볼, 10만볼트, 사이코키네시스

pokemon59 = Pokemon("나무킹", ["풀"], -1, -1, 70, 110, 75, 145, 85, 145, [copy.deepcopy(skill5), copy.deepcopy(skill50), copy.deepcopy(skill104), copy.deepcopy(skill133)]) #리프블레이드, 용의파동, 기합구슬, 암석봉인
pokemon60 = Pokemon("번치코", ["불꽃", "격투"], -1, -1, 80, 160, 80, 130, 80, 100, [copy.deepcopy(skill134), copy.deepcopy(skill70), copy.deepcopy(skill43), copy.deepcopy(skill15)]) #블레이즈킥, 스톤에지, 무릎차기, 번개펀치
pokemon61 = Pokemon("대짱이", ["불꽃", "격투"], -1, -1, 100, 150, 110, 95, 110, 70, [copy.deepcopy(skill32), copy.deepcopy(skill37), copy.deepcopy(skill72), copy.deepcopy(skill71)]) #탁류, 냉동빔, 하이드로펌프, 스톤샤워
pokemon62 = Pokemon("그라에나", ["악"], -1, -1, 70, 90, 70, 60, 60, 70, [copy.deepcopy(skill69), copy.deepcopy(skill87), copy.deepcopy(skill135), copy.deepcopy(skill100)]) #깨물어부수기, 섀도볼, 괴력, 기습
pokemon63 = Pokemon("직구리", ["노말"], -1, -1, 78, 70, 61, 50, 61, 100, [copy.deepcopy(skill136), copy.deepcopy(skill137), copy.deepcopy(skill111), copy.deepcopy(skill39)]) #은혜갚기, 배북, 씨폭탄, 섀도클로
pokemon64 = Pokemon("뷰티플라이", ["벌레", "비행"], -1, -1, 60, 70, 50, 100, 50, 65, [copy.deepcopy(skill78), copy.deepcopy(skill60), copy.deepcopy(skill92), copy.deepcopy(skill87)]) #벌레의야단법석, 에어컷터, 나비춤, 섀도볼
pokemon65 = Pokemon("독케일", ["벌레", "독"], -1, -1, 60, 50, 70, 50, 90, 65, [copy.deepcopy(skill92), copy.deepcopy(skill64), copy.deepcopy(skill138), copy.deepcopy(skill87)]) #나비춤, 맹독, 벌레의저항, 섀도볼
pokemon66 = Pokemon("로파파", ["풀", "물"], -1, -1, 80, 70, 70, 90, 100, 70, [copy.deepcopy(skill117), copy.deepcopy(skill49), copy.deepcopy(skill37), copy.deepcopy(skill104)]) #열탕, 에너지볼, 냉동빔, 기합구슬
pokemon67 = Pokemon("다탱구", ["풀", "악"], -1, -1, 90, 100, 60, 90, 60, 80, [copy.deepcopy(skill5), copy.deepcopy(skill100), copy.deepcopy(skill103), copy.deepcopy(skill139)]) #리프블레이드, 기습, 안다리걸기, 폭풍
pokemon68 = Pokemon("스왈로", ["노말", "비행"], -1, -1, 60, 85, 60, 75, 50, 125, [copy.deepcopy(skill136), copy.deepcopy(skill76), copy.deepcopy(skill66), copy.deepcopy(skill139)]) #은혜갚기, 브레이브버드, 강철날개, 폭풍
pokemon69 = Pokemon("페리퍼", ["물", "비행"], -1, -1, 60, 50, 100, 95, 70, 65, [copy.deepcopy(skill23), copy.deepcopy(skill37), copy.deepcopy(skill66), copy.deepcopy(skill139)]) #파도타기, 냉동빔, 강철날개, 폭풍
pokemon70 = Pokemon("가디안", ["물", "비행"], -1, -1, 68, 85, 65, 165, 135, 100, [copy.deepcopy(skill7), copy.deepcopy(skill82), copy.deepcopy(skill49), copy.deepcopy(skill125)]) #사이코키네시스, 10만볼트, 에너지볼, 파동탄
pokemon71 = Pokemon("비나방", ["벌레", "비행"], -1, -1, 70, 60, 62, 100, 82, 80, [copy.deepcopy(skill92), copy.deepcopy(skill74), copy.deepcopy(skill36), copy.deepcopy(skill78)]) #나비춤, 아쿠아제트, 에어슬래시, 벌레의야단법석
pokemon72 = Pokemon("버섯모", ["풀", "격투"], -1, -1, 60, 130, 80, 60, 60, 70, [copy.deepcopy(skill140), copy.deepcopy(skill111), copy.deepcopy(skill141), copy.deepcopy(skill71)]) #버섯포자, 씨폭탄, 스카이어퍼, 스톤샤워
pokemon73 = Pokemon("게을킹", ["노말"], -1, -1, 150, 160, 100, 95, 65, 100, [copy.deepcopy(skill53), copy.deepcopy(skill31), copy.deepcopy(skill61), copy.deepcopy(skill142)]) #게으름피우기, 파괴광선, 암해머, 속여때리기
pokemon74 = Pokemon("아이스크", ["벌레", "비행"], -1, -1, 61, 90, 45, 50, 50, 160, [copy.deepcopy(skill55), copy.deepcopy(skill114), copy.deepcopy(skill79), copy.deepcopy(skill143)]) #칼춤, 시저크로스, 제비반환, 메탈클로
pokemon75 = Pokemon("폭음룡", ["노말"], -1, -1, 104, 91, 63, 91, 73, 68, [copy.deepcopy(skill144), copy.deepcopy(skill145), copy.deepcopy(skill37), copy.deepcopy(skill104)]) #폭음파, 오버히트, 냉동빔, 기합구슬
pokemon76 = Pokemon("하리뭉", ["격투"], -1, -1, 144, 120, 60, 40, 60, 50, [copy.deepcopy(skill146), copy.deepcopy(skill56), copy.deepcopy(skill1), copy.deepcopy(skill70)]) #손바닥치기, 인파이트, 지진, 스톤에지
pokemon77 = Pokemon("마릴리", ["물"], -1, -1, 100, 50, 80, 60, 80, 50, [copy.deepcopy(skill25), copy.deepcopy(skill129), copy.deepcopy(skill12), copy.deepcopy(skill137)]) #폭포오르기, 엄청난힘, 냉동펀치, 배북
pokemon78 = Pokemon("델케티", ["노말"], -1, -1, 70, 65, 65, 55, 55, 90, [copy.deepcopy(skill147), copy.deepcopy(skill148), copy.deepcopy(skill149), copy.deepcopy(skill100)]) #나쁜음모, 전기자석파, 진흙폭탄, 기습
pokemon79 = Pokemon("깜까미", ["악", "고스트"], -1, -1, 50, 85, 125, 85, 115, 20, [copy.deepcopy(skill87), copy.deepcopy(skill119), copy.deepcopy(skill150), copy.deepcopy(skill133)]) #섀도볼, 이상한빛, 도깨비불, 암석봉인
pokemon80 = Pokemon("입치트", ["강철"], -1, -1, 50, 105, 125, 55, 95, 50, [copy.deepcopy(skill91), copy.deepcopy(skill100), copy.deepcopy(skill15), copy.deepcopy(skill151)]) #아이언헤드, 기습, 번개펀치, 불꽃엄니
pokemon81 = Pokemon("보스로라", ["강철", "바위"], -1, -1, 70, 140, 230, 60, 80, 50, [copy.deepcopy(skill91), copy.deepcopy(skill148), copy.deepcopy(skill70), copy.deepcopy(skill1)]) #아이언헤드, 전기자석파, 스톤에지, 지진
pokemon82 = Pokemon("요가램", ["격투", "에스퍼"], -1, -1, 60, 100, 85, 80, 85, 100, [copy.deepcopy(skill89), copy.deepcopy(skill43), copy.deepcopy(skill12), copy.deepcopy(skill71)]) #사념의박치기, 무릎차기, 냉동펀치, 스톤샤워
pokemon83 = Pokemon("썬더볼트", ["전기"], -1, -1, 70, 75, 80, 135, 80, 135, [copy.deepcopy(skill82), copy.deepcopy(skill34), copy.deepcopy(skill16), copy.deepcopy(skill69)]) #10만볼트, 화염방사, 시그널빔, 깨물어부수기
pokemon84 = Pokemon("플러시", ["전기"], -1, -1, 60, 50, 40, 85, 75, 95, [copy.deepcopy(skill152), copy.deepcopy(skill153), copy.deepcopy(skill24), copy.deepcopy(skill148)]) #플래시, 고속이동, 방전, 전기자석파
pokemon85 = Pokemon("마이농", ["전기"], -1, -1, 60, 40, 50, 75, 85, 95, [copy.deepcopy(skill152), copy.deepcopy(skill153), copy.deepcopy(skill24), copy.deepcopy(skill148)]) #플래시, 고속이동, 방전, 전기자석파
pokemon86 = Pokemon("네오비트", ["벌레"], -1, -1, 65, 47, 75, 73, 85, 85, [copy.deepcopy(skill78), copy.deepcopy(skill138), copy.deepcopy(skill30), copy.deepcopy(skill87)]) #벌레의야단법석, 벌레의저항, 전광석화, 섀도볼
pokemon87 = Pokemon("꿀꺽몬", ["독"], -1, -1, 100, 73, 83, 73, 83, 55, [copy.deepcopy(skill116), copy.deepcopy(skill154), copy.deepcopy(skill86), copy.deepcopy(skill88)]) #비축하기, 꿀꺽, 오물폭탄, 기가드레인
pokemon88 = Pokemon("샤크니아", ["물", "악"], -1, -1, 70, 140, 70, 110, 65, 105, [copy.deepcopy(skill69), copy.deepcopy(skill25), copy.deepcopy(skill51), copy.deepcopy(skill133)]) #깨물어부수기, 폭포오르기, 얼음엄니, 암석봉인
pokemon89 = Pokemon("고래왕", ["물"], -1, -1, 170, 90, 45, 90, 45, 60, [copy.deepcopy(skill155), copy.deepcopy(skill19), copy.deepcopy(skill1), copy.deepcopy(skill133)]) #헤수스파우팅, 눈보라, 지진, 암석봉인
pokemon90 = Pokemon("폭타", ["불꽃", "땅"], -1, -1, 70, 120, 100, 145, 105, 20, [copy.deepcopy(skill156), copy.deepcopy(skill157), copy.deepcopy(skill1), copy.deepcopy(skill73)]) #열풍, 분연, 지진, 러스터캐논
pokemon91 = Pokemon("코터스", ["불꽃"], -1, -1, 70, 85, 140, 85, 70, 20, [copy.deepcopy(skill34), copy.deepcopy(skill158), copy.deepcopy(skill159), copy.deepcopy(skill52)]) #화염방사, 철벽, 망각술, 하품
pokemon92 = Pokemon("피그킹", ["에스퍼"], -1, -1, 80, 45, 65, 90, 110, 80, [copy.deepcopy(skill7), copy.deepcopy(skill97), copy.deepcopy(skill49), copy.deepcopy(skill104)]) #사이코키네시스, 파워젬, 에너지볼, 기합구슬
pokemon93 = Pokemon("얼루기", ["노말"], -1, -1, 60, 60, 60, 60, 60, 60, [copy.deepcopy(skill160), copy.deepcopy(skill99), copy.deepcopy(skill100), copy.deepcopy(skill126)]) #흔들흔들댄스, 최면술, 기습, 물의파동
pokemon94 = Pokemon("플라이곤", ["땅", "드래곤"], -1, -1, 80, 100, 80, 80, 80, 100, [copy.deepcopy(skill1), copy.deepcopy(skill20), copy.deepcopy(skill71), copy.deepcopy(skill106)]) #지진, 드래곤클로, 스톤샤워, 공중날기
pokemon95 = Pokemon("밤선인", ["풀", "악"], -1, -1, 70, 115, 60, 115, 60, 55, [copy.deepcopy(skill55), copy.deepcopy(skill111), copy.deepcopy(skill129), copy.deepcopy(skill100)]) #칼춤, 씨폭탄, 엄청난힘, 기습
pokemon96 = Pokemon("파비코리", ["드래곤", "비행"], -1, -1, 75, 110, 110, 110, 105, 80, [copy.deepcopy(skill127), copy.deepcopy(skill20), copy.deepcopy(skill66), copy.deepcopy(skill37)]) #불새, 드래곤클로, 강철날개, 냉동빔
pokemon97 = Pokemon("쟝고", ["노말"], -1, -1, 73, 115, 60, 60, 60, 90, [copy.deepcopy(skill161), copy.deepcopy(skill55), copy.deepcopy(skill114), copy.deepcopy(skill56)]) #브레이크클로, 칼춤, 시저크로스, 인파이트
pokemon98 = Pokemon("세비퍼", ["독"], -1, -1, 73, 100, 60, 100, 60, 65, [copy.deepcopy(skill162), copy.deepcopy(skill163), copy.deepcopy(skill69), copy.deepcopy(skill164)]) #포이즌테일, 뱀눈초리, 깨물어부수기, 아쿠아테일
pokemon99 = Pokemon("루나톤", ["바위", "에스퍼"], -1, -1, 90, 55, 65, 95, 85, 70, [copy.deepcopy(skill37), copy.deepcopy(skill147), copy.deepcopy(skill97), copy.deepcopy(skill7)]) #냉동빔, 나쁜음모, 파워젬, 사이코키네시스
pokemon100 = Pokemon("솔록", ["바위", "에스퍼"], -1, -1, 90, 95, 85, 55, 65, 70, [copy.deepcopy(skill22), copy.deepcopy(skill70), copy.deepcopy(skill150), copy.deepcopy(skill89)]) #솔라빔, 스톤에지, 도깨비불, 사념의박치기
pokemon101 = Pokemon("메깅", ["물", "땅"], -1, -1, 110, 78, 73, 76, 71, 60, [copy.deepcopy(skill164), copy.deepcopy(skill1), copy.deepcopy(skill133), copy.deepcopy(skill59)]) #아쿠아테일, 지진, 암석봉인, 스파크
pokemon102 = Pokemon("가재장군", ["물", "악"], -1, -1, 63, 120, 85, 90, 55, 55, [copy.deepcopy(skill25), copy.deepcopy(skill69), copy.deepcopy(skill56), copy.deepcopy(skill114)]) #폭포오르기, 깨물어부수기, 인파이트, 시저크로스
pokemon103 = Pokemon("점토도리", ["땅", "에스퍼"], -1, -1, 60, 70, 105, 70, 120, 75, [copy.deepcopy(skill165), copy.deepcopy(skill166), copy.deepcopy(skill124), copy.deepcopy(skill1)]) #진흙뿌리기, 환상빔, 원시의힘, 지진
pokemon104 = Pokemon("릴리요", ["바위", "풀"], -1, -1, 86, 81, 97, 81, 107, 43, [copy.deepcopy(skill70), copy.deepcopy(skill49), copy.deepcopy(skill167), copy.deepcopy(skill1)]) #스톤에지, 에너지볼, 오물웨이브, 지진
pokemon105 = Pokemon("아말도", ["바위", "벌레"], -1, -1, 75, 125, 100, 70, 80, 45, [copy.deepcopy(skill168), copy.deepcopy(skill169), copy.deepcopy(skill74), copy.deepcopy(skill114)]) #록커트, 록블라스트, 아쿠아제트, 시저크로스
pokemon106 = Pokemon("밀로틱", ["물"], -1, -1, 95, 60, 79, 100, 125, 81, [copy.deepcopy(skill72), copy.deepcopy(skill37), copy.deepcopy(skill50), copy.deepcopy(skill170)]) #하이드로펌프, 냉동빔, 용의파동, HP회복
pokemon107 = Pokemon("캐스퐁", ["노말"], -1, -1, 70, 70, 70, 70, 70, 70, [copy.deepcopy(skill171), copy.deepcopy(skill34), copy.deepcopy(skill4), copy.deepcopy(skill19)]) #웨더볼, 화염방사, 번개, 눈보라
pokemon108 = Pokemon("켈리몬", ["노말"], -1, -1, 60, 90, 70, 60, 120, 40, [copy.deepcopy(skill100), copy.deepcopy(skill172), copy.deepcopy(skill170), copy.deepcopy(skill113)]) #기습, 속임수, HP회복, 드레인펀치
pokemon109 = Pokemon("다크펫", ["고스트"], -1, -1, 64, 165, 75, 93, 83, 75, [copy.deepcopy(skill39), copy.deepcopy(skill3), copy.deepcopy(skill82), copy.deepcopy(skill136)]) #섀도클로, 더스트슈트, 10만볼트, 은혜갚기
pokemon110 = Pokemon("트로피우스", ["풀", "비행"], -1, -1, 99, 68, 83, 72, 87, 51, [copy.deepcopy(skill173), copy.deepcopy(skill36), copy.deepcopy(skill120), copy.deepcopy(skill174)]) #안개제거, 에어슬래시, 리프스톰, 풀베기
pokemon111 = Pokemon("앱솔", ["악"], -1, -1, 65, 150, 60, 115, 60, 115, [copy.deepcopy(skill100), copy.deepcopy(skill89), copy.deepcopy(skill129), copy.deepcopy(skill175)]) #기습, 사념의박치기, 엄청난힘, 멸망의노래
pokemon112 = Pokemon("마자용", ["에스퍼"], -1, -1, 190, 33, 58, 33, 58, 33, [copy.deepcopy(skill159), copy.deepcopy(skill176), copy.deepcopy(skill177), copy.deepcopy(skill178)]) #망각술, 신비의부적, 카운터, 미러코트
pokemon113 = Pokemon("얼음귀신", ["얼음"], -1, -1, 80, 120, 80, 120, 80, 100, [copy.deepcopy(skill19), copy.deepcopy(skill1), copy.deepcopy(skill179), copy.deepcopy(skill180)]) #눈보라, 지진, 싸라기눈, 절대영도
pokemon114 = Pokemon("씨카이저", ["얼음", "물"], -1, -1, 110, 80, 90, 95, 90, 65, [copy.deepcopy(skill37), copy.deepcopy(skill23), copy.deepcopy(skill1), copy.deepcopy(skill16)]) #냉동빔, 파도타기, 지진, 시그널빔
pokemon115 = Pokemon("헌테일", ["물"], -1, -1, 55, 104, 105, 94, 75, 52, [copy.deepcopy(skill100), copy.deepcopy(skill51), copy.deepcopy(skill181), copy.deepcopy(skill117)]) #기습, 얼음엄니, 뛰어오르기, 열탕
pokemon116 = Pokemon("분홍장이", ["물"], -1, -1, 55, 84, 105, 114, 75, 52, [copy.deepcopy(skill23), copy.deepcopy(skill37), copy.deepcopy(skill7), copy.deepcopy(skill87)]) #파도타기, 냉동빔, 사이코키네시스, 섀도볼
pokemon117 = Pokemon("시라칸", ["물", "바위"], -1, -1, 100, 90, 130, 45, 65, 55, [copy.deepcopy(skill90), copy.deepcopy(skill168), copy.deepcopy(skill164), copy.deepcopy(skill26)]) #양날박치기, 록커트, 아쿠아테일, 웨이브태클
pokemon118 = Pokemon("사랑동이", ["물"], -1, -1, 43, 30, 55, 40, 65, 97, [copy.deepcopy(skill72), copy.deepcopy(skill9), copy.deepcopy(skill182), copy.deepcopy(skill64)]) #하이드로펌프, 애교부리기, 천사의키스, 맹독
pokemon119 = Pokemon("보만다", ["드래곤", "비행"], -1, -1, 95, 145, 130, 120, 90, 120, [copy.deepcopy(skill75), copy.deepcopy(skill2), copy.deepcopy(skill109), copy.deepcopy(skill183)]) #이판사판태클, 불대문자, 용성군, 용의춤
pokemon120 = Pokemon("메타그로스", ["강철", "에스퍼"], -1, -1, 80, 145, 150, 105, 110, 110, [copy.deepcopy(skill184), copy.deepcopy(skill89), copy.deepcopy(skill12), copy.deepcopy(skill1)]) #코멧펀치, 사념의박치기, 냉동펀치, 지진
pokemon121 = Pokemon("메가니움", ["풀"], -1, -1, 80, 82, 100, 83, 100, 80, [copy.deepcopy(skill177), copy.deepcopy(skill123), copy.deepcopy(skill120), copy.deepcopy(skill110)]) #카운터, 역린, 리프스톰, 누르기
pokemon122 = Pokemon("블레이범", ["불꽃"], -1, -1, 73, 84, 78, 119, 85, 95, [copy.deepcopy(skill185), copy.deepcopy(skill186), copy.deepcopy(skill104), copy.deepcopy(skill1)]) #불사르기, 분화, 기합구슬, 지진
pokemon123 = Pokemon("장크로다일", ["물"], -1, -1, 85, 105, 100, 79, 83, 78, [copy.deepcopy(skill25), copy.deepcopy(skill55), copy.deepcopy(skill12), copy.deepcopy(skill71)]) #폭포오르기, 칼춤, 냉동펀치, 스톤샤워
pokemon124 = Pokemon("야부엉", ["노말", "비행"], -1, -1, 100, 50, 50, 86, 96, 70, [copy.deepcopy(skill80), copy.deepcopy(skill36), copy.deepcopy(skill7), copy.deepcopy(skill156)]) #하이퍼보이스, 사이코키네시스, 기습, 열풍
pokemon125 = Pokemon("레디안", ["벌레", "비행"], -1, -1, 55, 35, 50, 55, 110, 85, [copy.deepcopy(skill78), copy.deepcopy(skill12), copy.deepcopy(skill15), copy.deepcopy(skill173)]) #벌레의야단법석, 냉동펀치, 번개펀치, 안개제거
pokemon126 = Pokemon("아리아도스", ["벌레", "독"], -1, -1, 70, 90, 70, 60, 70, 40, [copy.deepcopy(skill86), copy.deepcopy(skill16), copy.deepcopy(skill188), copy.deepcopy(skill132)]) #오물폭탄, 시그널빔, 구멍파기, 야습
pokemon127 = Pokemon("다꼬리", ["노말"], -1, -1, 85, 76, 64, 45, 55, 90, [copy.deepcopy(skill187), copy.deepcopy(skill136), copy.deepcopy(skill100), copy.deepcopy(skill14)]) #정리정돈, 은혜갚기, 기습, 박치기
pokemon128 = Pokemon("크로뱃", ["독", "비행"], -1, -1, 85, 90, 80, 70, 80, 130, [copy.deepcopy(skill112), copy.deepcopy(skill60), copy.deepcopy(skill87), copy.deepcopy(skill88)]) #크로스포이즌, 에어컷터, 섀도볼, 기가드레인
pokemon129 = Pokemon("랜턴", ["물", "전기"], -1, -1, 125, 58, 58, 76, 76, 67, [copy.deepcopy(skill118), copy.deepcopy(skill23), copy.deepcopy(skill24), copy.deepcopy(skill37)]) #비바라기, 파도타기, 방전, 냉동빔
pokemon130 = Pokemon("픽시", ["노말"], -1, -1, 95, 70, 73, 95, 90, 60, [copy.deepcopy(skill80), copy.deepcopy(skill82), copy.deepcopy(skill28), copy.deepcopy(skill37)]) #하이퍼보이스, 10만볼트, 매지컬리프, 냉동빔
pokemon131 = Pokemon("푸크린", ["노말"], -1, -1, 140, 70, 45, 85, 50, 45, [copy.deepcopy(skill80), copy.deepcopy(skill189), copy.deepcopy(skill104), copy.deepcopy(skill7)]) #하이퍼보이스, 노래하기, 기합구슬, 사이코키네시스
pokemon132 = Pokemon("네이티오", ["에스퍼", "비행"], -1, -1, 65, 75, 70, 95, 70, 95, [copy.deepcopy(skill156), copy.deepcopy(skill36), copy.deepcopy(skill87), copy.deepcopy(skill7)]) #열풍, 에어슬래시, 섀도볼, 사이코키네시스
pokemon133 = Pokemon("전룡", ["전기"], -1, -1, 90, 95, 105, 165, 110, 45, [copy.deepcopy(skill82), copy.deepcopy(skill97), copy.deepcopy(skill50), copy.deepcopy(skill104)]) #10만볼트, 파워젬, 용의파동, 기합구슬
pokemon134 = Pokemon("아르코", ["풀"], -1, -1, 75, 80, 95, 90, 100, 50, [copy.deepcopy(skill85), copy.deepcopy(skill49), copy.deepcopy(skill86), copy.deepcopy(skill113)]) #꽃잎댄스, 에너지볼, 오물폭탄, 드레인펀치
pokemon135 = Pokemon("왕구리", ["물"], -1, -1, 90, 75, 75, 90, 100, 70, [copy.deepcopy(skill175), copy.deepcopy(skill72), copy.deepcopy(skill37), copy.deepcopy(skill104)]) #멸망의노래, 하이드로펌프, 냉동빔, 기합구슬
pokemon136 = Pokemon("솜솜코", ["풀", "비행"], -1, -1, 75, 55, 70, 55, 95, 110, [copy.deepcopy(skill190), copy.deepcopy(skill191), copy.deepcopy(skill111), copy.deepcopy(skill192)]) #수면가루, 순풍, 씨폭탄, 저리가루
pokemon137 = Pokemon("누오", ["물", "땅"], -1, -1, 95, 85, 85, 65, 65, 35, [copy.deepcopy(skill164), copy.deepcopy(skill1), copy.deepcopy(skill70), copy.deepcopy(skill117)]) #아쿠아테일, 지진, 스톤에지, 열탕
pokemon138 = Pokemon("에브이", ["에스퍼"], -1, -1, 65, 65, 60, 130, 95, 110, [copy.deepcopy(skill7), copy.deepcopy(skill87), copy.deepcopy(skill16), copy.deepcopy(skill193)]) #사이코키네시스, 섀도볼, 시그널빔, 아침햇살
pokemon139 = Pokemon("블래키", ["악"], -1, -1, 95, 65, 110, 60, 130, 65, [copy.deepcopy(skill108), copy.deepcopy(skill84), copy.deepcopy(skill194), copy.deepcopy(skill195)]) #악의파동, 아이언테일, 달빛, 바크아웃
pokemon140 = Pokemon("야도킹", ["물", "에스퍼"], -1, -1, 95, 75, 80, 100, 110, 30, [copy.deepcopy(skill23), copy.deepcopy(skill7), copy.deepcopy(skill37), copy.deepcopy(skill34)]) #파도타기, 사이코키네시스, 냉동빔, 화염방사
pokemon141 = Pokemon("안농", ["에스퍼"], -1, -1, 48, 72, 48, 72, 48, 48, [copy.deepcopy(skill196)]) #잠재파워
pokemon142 = Pokemon("키링키", ["노말", "에스퍼"], -1, -1, 70, 80, 65, 90, 65, 85, [copy.deepcopy(skill197), copy.deepcopy(skill49), copy.deepcopy(skill69), copy.deepcopy(skill1)]) #트윈빔, 에너지볼, 깨물어부수기, 지진
pokemon143 = Pokemon("쏘콘", ["벌레", "강철"], -1, -1, 75, 90, 140, 60, 60, 40, [copy.deepcopy(skill65), copy.deepcopy(skill81), copy.deepcopy(skill169), copy.deepcopy(skill10)]) #자이로볼, 벌레먹기, 록블라스트, 독찌르기
pokemon144 = Pokemon("노고치", ["노말"], -1, -1, 100, 70, 70, 65, 65, 45, [copy.deepcopy(skill163), copy.deepcopy(skill14), copy.deepcopy(skill71), copy.deepcopy(skill198)]) #뱀눈초리, 박치기, 스톤샤워, 따라가때리기
pokemon145 = Pokemon("강철톤", ["강철", "땅"], -1, -1, 75, 125, 230, 55, 95, 30, [copy.deepcopy(skill84), copy.deepcopy(skill1), copy.deepcopy(skill70), copy.deepcopy(skill69)]) #아이언테일, 지진, 스톤에지, 깨물어부수기
pokemon146 = Pokemon("그랑블루", ["노말"], -1, -1, 90, 120, 75, 60, 60, 45, [copy.deepcopy(skill136), copy.deepcopy(skill123), copy.deepcopy(skill51), copy.deepcopy(skill70)]) #은혜갚기, 역린, 얼음엄니, 스톤에지
pokemon147 = Pokemon("침바루", ["물", "독"], -1, -1, 65, 95, 85, 55, 55, 85, [copy.deepcopy(skill199), copy.deepcopy(skill200), copy.deepcopy(skill74), copy.deepcopy(skill55)]) #독침천발, 작아지기, 아쿠아제트, 칼춤
pokemon148 = Pokemon("핫삼", ["벌레", "강철"], -1, -1, 70, 150, 140, 65, 100, 75, [copy.deepcopy(skill143), copy.deepcopy(skill81), copy.deepcopy(skill201), copy.deepcopy(skill55)]) #메탈클로, 벌레먹기, 날개치기, 칼춤
pokemon149 = Pokemon("단단지", ["벌레", "바위"], -1, -1, 20, 10, 230, 10, 230, 5, [copy.deepcopy(skill202), copy.deepcopy(skill70), copy.deepcopy(skill203), copy.deepcopy(skill81)]) #숲의저주, 스톤에지, 파워트릭, 벌레먹기
pokemon150 = Pokemon("헤라크로스", ["벌레", "격투"], -1, -1, 80, 185, 115, 40, 105, 75, [copy.deepcopy(skill122), copy.deepcopy(skill56), copy.deepcopy(skill1), copy.deepcopy(skill39)]) #메가혼, 인파이트, 지진, 섀도클로
pokemon151 = Pokemon("링곰", ["노말"], -1, -1, 90, 130, 75, 75, 75, 55, [copy.deepcopy(skill204), copy.deepcopy(skill56), copy.deepcopy(skill1), copy.deepcopy(skill205)]) #객기, 인파이트, 지진, 벌크업
pokemon152 = Pokemon("마그카르고", ["불꽃", "바위"], -1, -1, 60, 50, 120, 90, 80, 30, [copy.deepcopy(skill34), copy.deepcopy(skill71), copy.deepcopy(skill52), copy.deepcopy(skill158)]) #화염방사, 스톤샤워, 하품, 철벽
pokemon153 = Pokemon("코산호", ["물", "바위"], -1, -1, 65, 55, 95, 65, 95, 35, [copy.deepcopy(skill178), copy.deepcopy(skill97), copy.deepcopy(skill23), copy.deepcopy(skill35)]) #미러코트, 파워젬, 파도타기, 대지의힘
pokemon154 = Pokemon("대포무노", ["물"], -1, -1, 75, 105, 75, 105, 75, 45, [copy.deepcopy(skill206), copy.deepcopy(skill62), copy.deepcopy(skill37), copy.deepcopy(skill73)]) #대포무노포, 아쿠아브레이크, 냉동빔, 러스터캐논
pokemon155 = Pokemon("딜리버드", ["얼음", "비행"], -1, -1, 45, 55, 45, 65, 45, 75, [copy.deepcopy(skill207), copy.deepcopy(skill33), copy.deepcopy(skill196), copy.deepcopy(skill128)]) #프리젠트, 회전부리, 잠재파워, 얼음뭉치
pokemon156 = Pokemon("무장조", ["강철", "비행"], -1, -1, 65, 80, 140, 40, 70, 70, [copy.deepcopy(skill66), copy.deepcopy(skill33), copy.deepcopy(skill71), copy.deepcopy(skill55)]) #강철날개, 회전부리, 스톤샤워, 칼춤
pokemon157 = Pokemon("헬가", ["악", "불꽃"], -1, -1, 75, 90, 90, 140, 90, 115, [copy.deepcopy(skill34), copy.deepcopy(skill108), copy.deepcopy(skill86), copy.deepcopy(skill83)]) #화염방사, 악의파동, 오물폭탄, 번개엄니
pokemon158 = Pokemon("킹드라", ["물", "드래곤"], -1, -1, 75, 95, 95, 95, 95, 85, [copy.deepcopy(skill62), copy.deepcopy(skill50), copy.deepcopy(skill73), copy.deepcopy(skill139)]) #아쿠아브레이크, 용의파동, 러스터캐논, 폭풍
pokemon159 = Pokemon("코리갑", ["땅"], -1, -1, 90, 120, 120, 60, 60, 50, [copy.deepcopy(skill1), copy.deepcopy(skill71), copy.deepcopy(skill151), copy.deepcopy(skill111)]) #지진, 스톤샤워, 불꽃엄니, 씨폭탄
pokemon160 = Pokemon("노라키", ["노말"], -1, -1, 73, 95, 62, 85, 65, 85, [copy.deepcopy(skill208), copy.deepcopy(skill99), copy.deepcopy(skill209), copy.deepcopy(skill210)]) #배리어러시, 최면술, 짓밟기, 와일드볼트
pokemon161 = Pokemon("루브도", ["노말"], -1, -1, 55, 20, 35, 20, 45, 75, [copy.deepcopy(skill140), copy.deepcopy(skill54), copy.deepcopy(skill180), copy.deepcopy(skill92)]) #버섯포자, 신속, 절대영도, 나비춤
pokemon162 = Pokemon("카포에라", ["격투"], -1, -1, 50, 95, 95, 35, 110, 70, [copy.deepcopy(skill211), copy.deepcopy(skill70), copy.deepcopy(skill65), copy.deepcopy(skill177)]) #버섯포자, 스톤에지, 자이로볼, 카운터
pokemon163 = Pokemon("루주라", ["노말"], -1, -1, 65, 50, 35, 115, 95, 95, [copy.deepcopy(skill212), copy.deepcopy(skill37), copy.deepcopy(skill7), copy.deepcopy(skill49)]) #악마의키스, 냉동빔, 사이코키네시스, 에너지볼
pokemon164 = Pokemon("밀탱크", ["노말"], -1, -1, 95, 80, 105, 40, 70, 100, [copy.deepcopy(skill213), copy.deepcopy(skill136), copy.deepcopy(skill89), copy.deepcopy(skill1)]) #우유마시기, 은혜갚기, 사념의박치기, 지진
pokemon165 = Pokemon("마기라스", ["바위", "악"], -1, -1, 100, 164, 150, 95, 120, 71, [copy.deepcopy(skill71), copy.deepcopy(skill69), copy.deepcopy(skill91), copy.deepcopy(skill1)]) #스톤샤워, 깨물어부수기, 아이언헤드, 지진
pokemon166 = Pokemon("이상해꽃", ["풀", "독"], -1, -1, 80, 100, 123, 122, 120, 80, [copy.deepcopy(skill88), copy.deepcopy(skill49), copy.deepcopy(skill64), copy.deepcopy(skill86)]) #기가드레인, 에너지볼, 맹독, 오물폭탄
pokemon167 = Pokemon("리자몽", ["불꽃", "비행"], -1, -1, 78, 104, 78, 159, 115, 100, [copy.deepcopy(skill36), copy.deepcopy(skill34), copy.deepcopy(skill50), copy.deepcopy(skill22)]) #에어슬래시, 화염방사, 용의파동, 솔라빔
pokemon168 = Pokemon("거북왕", ["물"], -1, -1, 79, 103, 120, 135, 115, 78, [copy.deepcopy(skill23), copy.deepcopy(skill37), copy.deepcopy(skill73), copy.deepcopy(skill108)]) #파도타기, 냉동빔, 러스터캐논, 악의파동
pokemon168 = Pokemon("버터플", ["벌레", "비행"], -1, -1, 60, 45, 50, 90, 80, 70, [copy.deepcopy(skill190), copy.deepcopy(skill81), copy.deepcopy(skill191), copy.deepcopy(skill7)]) #수면가루, 벌레먹기, 순풍, 사이코키네시스
pokemon169 = Pokemon("독침붕", ["벌레", "독"], -1, -1, 65, 150, 40, 15, 80, 145, [copy.deepcopy(skill114), copy.deepcopy(skill81), copy.deepcopy(skill191), copy.deepcopy(skill45)]) #시저크로스, 벌레먹기, 순풍, 깨트리기
pokemon170 = Pokemon("피죤투", ["노말", "비행"], -1, -1, 83, 80, 80, 135, 80, 121, [copy.deepcopy(skill106), copy.deepcopy(skill139), copy.deepcopy(skill156), copy.deepcopy(skill36)]) #공중날기, 폭풍, 열풍, 에어슬래시
pokemon171 = Pokemon("레트라", ["노말"], -1, -1, 55, 81, 60, 50, 70, 97, [copy.deepcopy(skill55), copy.deepcopy(skill100), copy.deepcopy(skill75), copy.deepcopy(skill188)]) #칼춤, 기습, 이판사판태클, 구멍파기
pokemon172 = Pokemon("깨비드릴조", ["노말", "비행"], -1, -1, 65, 90, 65, 61, 61, 100, [copy.deepcopy(skill33), copy.deepcopy(skill131), copy.deepcopy(skill30), copy.deepcopy(skill66)]) #회전부리, 트라이어택, 전광석화, 강철날개
pokemon173 = Pokemon("아보크", ["독"], -1, -1, 60, 95, 69, 65, 79, 80, [copy.deepcopy(skill163), copy.deepcopy(skill3), copy.deepcopy(skill51), copy.deepcopy(skill69)]) #뱀눈초리, 더스트슈트, 얼음엄니, 깨물어부수기
pokemon174 = Pokemon("라이츄", ["전기"], -1, -1, 60, 85, 50, 95, 85, 110, [copy.deepcopy(skill8), copy.deepcopy(skill82), copy.deepcopy(skill23), copy.deepcopy(skill45)]) #볼트태클, 10만볼트, 파도타기, 깨트리기
pokemon175 = Pokemon("고지", ["땅"], -1, -1, 75, 100, 110, 45, 55, 65, [copy.deepcopy(skill1), copy.deepcopy(skill65), copy.deepcopy(skill39), copy.deepcopy(skill79)]) #지진, 자이로볼, 섀도클로, 제비반환
pokemon176 = Pokemon("니드퀸", ["독", "땅"], -1, -1, 90, 92, 87, 75, 85, 76, [copy.deepcopy(skill1), copy.deepcopy(skill86), copy.deepcopy(skill34), copy.deepcopy(skill39)]) #지진, 오물폭탄, 화염방사, 섀도클로
pokemon177 = Pokemon("니드킹", ["독", "땅"], -1, -1, 81, 102, 77, 85, 75, 85, [copy.deepcopy(skill1), copy.deepcopy(skill86), copy.deepcopy(skill34), copy.deepcopy(skill39)]) #지진, 오물폭탄, 화염방사, 섀도클로
pokemon178 = Pokemon("나인테일", ["불꽃"], -1, -1, 81, 102, 77, 85, 75, 85, [copy.deepcopy(skill87), copy.deepcopy(skill22), copy.deepcopy(skill34), copy.deepcopy(skill214)]) #섀도볼, 솔라빔, 화염방사, 사이코쇼크
pokemon179 = Pokemon("라플레시아", ["풀", "독"], -1, -1, 81, 102, 77, 85, 75, 85, [copy.deepcopy(skill49), copy.deepcopy(skill86), copy.deepcopy(skill113), copy.deepcopy(skill215)]) #에너지볼, 오물폭탄, 드레인펀치, 꽃보라
pokemon180 = Pokemon("파라섹트", ["벌레", "풀"], -1, -1, 60, 95, 80, 60, 80, 30, [copy.deepcopy(skill140), copy.deepcopy(skill111), copy.deepcopy(skill114), copy.deepcopy(skill177)]) #버섯포자, 씨폭탄, 시저크로스, 카운터
pokemon181 = Pokemon("도나리", ["벌레", "독"], -1, -1, 70, 65, 60, 90, 75, 90, [copy.deepcopy(skill49), copy.deepcopy(skill86), copy.deepcopy(skill7), copy.deepcopy(skill78)]) #에너지볼, 오물폭탄, 사이코키네시스, 벌레의야단법석
pokemon182 = Pokemon("닥트리오", ["땅"], -1, -1, 35, 100, 50, 50, 70, 120, [copy.deepcopy(skill1), copy.deepcopy(skill29), copy.deepcopy(skill70), copy.deepcopy(skill79)]) #지진, 깜짝베기, 스톤에지, 제비반환
pokemon183 = Pokemon("페르시온", ["노말"], -1, -1, 65, 60, 60, 75, 65, 115, [copy.deepcopy(skill147), copy.deepcopy(skill216), copy.deepcopy(skill108), copy.deepcopy(skill79)]) #나쁜음모, 스피드스타, 악의파동, 제비반환
pokemon184 = Pokemon("골덕", ["물"], -1, -1, 80, 82, 78, 95, 80, 85, [copy.deepcopy(skill72), copy.deepcopy(skill7), copy.deepcopy(skill104), copy.deepcopy(skill37)]) #하이드로펌프, 사이코키네시스, 기합구슬, 냉동빔
pokemon185 = Pokemon("성원숭", ["격투"], -1, -1, 65, 105, 60, 60, 70, 95, [copy.deepcopy(skill56), copy.deepcopy(skill123), copy.deepcopy(skill10), copy.deepcopy(skill103)]) #인파이트, 역린, 독찌르기, 안다리걸기
pokemon186 = Pokemon("윈디", ["불꽃"], -1, -1, 90, 110, 80, 100, 80, 95, [copy.deepcopy(skill54), copy.deepcopy(skill63), copy.deepcopy(skill69), copy.deepcopy(skill50)]) #신속, 플레어드라이브, 깨물어부수기, 용의파동
pokemon187 = Pokemon("강챙이", ["물", "격투"], -1, -1, 90, 110, 80, 100, 80, 95, [copy.deepcopy(skill25), copy.deepcopy(skill45), copy.deepcopy(skill12), copy.deepcopy(skill110)]) #폭포오르기, 깨트리기, 냉동펀치, 누르기
pokemon188 = Pokemon("후딘", ["에스퍼"], -1, -1, 55, 50, 65, 175, 105, 150, [copy.deepcopy(skill217), copy.deepcopy(skill7), copy.deepcopy(skill104), copy.deepcopy(skill87)]) #숟가락휘기, 사이코키네시스, 기합구슬, 섀도볼
pokemon189 = Pokemon("괴력몬", ["격투"], -1, -1, 90, 130, 80, 65, 85, 55, [copy.deepcopy(skill218), copy.deepcopy(skill15), copy.deepcopy(skill11), copy.deepcopy(skill12)]) #폭발펀치, 번개펀치, 불꽃펀치, 냉동펀치
pokemon190 = Pokemon("우츠보트", ["풀", "독"], -1, -1, 80, 105, 65, 100, 70, 70, [copy.deepcopy(skill5), copy.deepcopy(skill10), copy.deepcopy(skill136), copy.deepcopy(skill120)]) #리프블레이드, 독찌르기, 은혜갚기, 리프스톰
pokemon191 = Pokemon("독파리", ["물", "독"], -1, -1, 80, 70, 65, 80, 120, 100, [copy.deepcopy(skill23), copy.deepcopy(skill86), copy.deepcopy(skill37), copy.deepcopy(skill88)]) #파도타기, 오물폭탄, 냉동빔, 기가드레인
pokemon192 = Pokemon("딱구리", ["바위", "땅"], -1, -1, 80, 120, 130, 55, 65, 45, [copy.deepcopy(skill1), copy.deepcopy(skill70), copy.deepcopy(skill75), copy.deepcopy(skill65)]) #지진, 스톤에지, 이판사판태클, 자이로볼
pokemon193 = Pokemon("날쌩마", ["불꽃"], -1, -1, 65, 100, 70, 80, 80, 105, [copy.deepcopy(skill63), copy.deepcopy(skill122), copy.deepcopy(skill84), copy.deepcopy(skill210)]) #플레어드라이브, 메가혼, 아이언테일, 와일드볼트
pokemon194 = Pokemon("야도란", ["물", "에스퍼"], -1, -1, 95, 75, 180, 130, 80, 30, [copy.deepcopy(skill159), copy.deepcopy(skill64), copy.deepcopy(skill158), copy.deepcopy(skill23)]) #망각술, 맹독, 철벽, 파도타기
pokemon195 = Pokemon("파오리", ["노말", "비행"], -1, -1, 52, 95, 55, 58, 62, 55, [copy.deepcopy(skill76), copy.deepcopy(skill5), copy.deepcopy(skill153), copy.deepcopy(skill29)]) #브레이브버드, 리프블레이드, 고속이동, 깜짝베기
pokemon196 = Pokemon("두트리오", ["노말", "비행"], -1, -1, 60, 110, 70, 60, 60, 110, [copy.deepcopy(skill136), copy.deepcopy(skill130), copy.deepcopy(skill33), copy.deepcopy(skill66)]) #은혜갚기, 난동부리기, 회전부리, 강철날개
pokemon197 = Pokemon("쥬레곤", ["물", "얼음"], -1, -1, 90, 70, 80, 70, 95, 70, [copy.deepcopy(skill74), copy.deepcopy(skill116), copy.deepcopy(skill180), copy.deepcopy(skill37)]) #아쿠아제트, 비축하기, 절대영도, 냉동빔
pokemon198 = Pokemon("질뻐기", ["독"], -1, -1, 105, 105, 75, 65, 100, 50, [copy.deepcopy(skill3), copy.deepcopy(skill69), copy.deepcopy(skill45), copy.deepcopy(skill71)]) #더스트슈트, 깨물어부수기, 깨트리기, 스톤샤워
pokemon199 = Pokemon("파르셀", ["물", "얼음"], -1, -1, 50, 95, 180, 85, 45, 70, [copy.deepcopy(skill62), copy.deepcopy(skill37), copy.deepcopy(skill10), copy.deepcopy(skill102)]) #아쿠아브레이크, 냉동빔, 독찌르기, 보복
pokemon200 = Pokemon("팬텀", ["고스트", "독"], -1, -1, 60, 65, 80, 170, 95, 130, [copy.deepcopy(skill87), copy.deepcopy(skill49), copy.deepcopy(skill82), copy.deepcopy(skill167)]) #섀도볼, 에너지볼, 10만볼트, 오물웨이브
pokemon201 = Pokemon("슬리퍼", ["에스퍼"], -1, -1, 85, 73, 70, 73, 115, 67, [copy.deepcopy(skill7), copy.deepcopy(skill89), copy.deepcopy(skill16), copy.deepcopy(skill87)]) #사이코키네시스, 사념의박치기, 시그널빔, 섀도볼
pokemon202 = Pokemon("킹크랩", ["물"], -1, -1, 55, 130, 115, 50, 50, 75, [copy.deepcopy(skill219), copy.deepcopy(skill129), copy.deepcopy(skill55), copy.deepcopy(skill114)]) #집게해머, 엄청난힘, 칼춤, 시저크로스
pokemon203 = Pokemon("붐볼", ["전기"], -1, -1, 60, 50, 70, 80, 80, 150, [copy.deepcopy(skill220), copy.deepcopy(skill82), copy.deepcopy(skill178), copy.deepcopy(skill5)]) #싫은소리, 10만볼트, 미러코트, 볼부비부비
pokemon204 = Pokemon("나시", ["풀", "에스퍼"], -1, -1, 95, 95, 85, 125, 75, 55, [copy.deepcopy(skill22), copy.deepcopy(skill7), copy.deepcopy(skill86), copy.deepcopy(skill27)]) #솔라빔, 사이코키네시스, 오물폭탄, 쾌청
pokemon205 = Pokemon("텅구리", ["땅"], -1, -1, 60, 80, 110, 50, 80, 45, [copy.deepcopy(skill137), copy.deepcopy(skill1), copy.deepcopy(skill63), copy.deepcopy(skill196)]) #배북, 지진, 플레어드라이브, 잠재파워
pokemon206 = Pokemon("시라소몬", ["격투"], -1, -1, 50, 120, 53, 35, 110, 87, [copy.deepcopy(skill43), copy.deepcopy(skill44), copy.deepcopy(skill133), copy.deepcopy(skill1)]) #무릎차기, 마하펀치, 암석봉인, 지진
pokemon207 = Pokemon("홍수몬", ["격투"], -1, -1, 50, 105, 79, 35, 110, 76, [copy.deepcopy(skill57), copy.deepcopy(skill44), copy.deepcopy(skill15), copy.deepcopy(skill158)]) #불릿펀치, 마하펀치, 번개펀치, 철벽
pokemon208 = Pokemon("또도가스", ["독"], -1, -1, 65, 90, 120, 85, 70, 60, [copy.deepcopy(skill150), copy.deepcopy(skill86), copy.deepcopy(skill2), copy.deepcopy(skill108)]) #도깨비불, 오물폭탄, 불대문자, 악의파동
pokemon209 = Pokemon("캥카", ["노말"], -1, -1, 105, 125, 100, 60, 100, 100, [copy.deepcopy(skill221), copy.deepcopy(skill110), copy.deepcopy(skill79), copy.deepcopy(skill84)]) #잼잼펀치, 누르기, 제비반환, 아이언테일
pokemon210 = Pokemon("왕콘치", ["물"], -1, -1, 80, 92, 65, 65, 80, 68, [copy.deepcopy(skill25), copy.deepcopy(skill122), copy.deepcopy(skill153), copy.deepcopy(skill222)]) #폭포오르기, 메가혼, 고속이동, 뿔드릴
pokemon211 = Pokemon("아쿠스타", ["물", "에스퍼"], -1, -1, 60, 75, 85, 100, 85, 115, [copy.deepcopy(skill72), copy.deepcopy(skill7), copy.deepcopy(skill82), copy.deepcopy(skill19)]) #하이드로펌프, 사이코키네시스, 10만볼트, 눈보라
pokemon212 = Pokemon("쁘사이저", ["벌레", "비행"], -1, -1, 65, 155, 120, 65, 90, 105, [copy.deepcopy(skill114), copy.deepcopy(skill129), copy.deepcopy(skill1), copy.deepcopy(skill71)]) #시저크로스, 엄청난힘, 지진, 스톤샤워
pokemon213 = Pokemon("켄타로스", ["노말"], -1, -1, 75, 100, 95, 40, 70, 110, [copy.deepcopy(skill110), copy.deepcopy(skill210), copy.deepcopy(skill1), copy.deepcopy(skill122)]) #누르기, 와일드볼트, 지진, 메가혼
pokemon214 = Pokemon("갸라도스", ["물", "비행"], -1, -1, 95, 155, 109, 70, 130, 81, [copy.deepcopy(skill164), copy.deepcopy(skill69), copy.deepcopy(skill181), copy.deepcopy(skill183)]) #아쿠아테일, 깨물어부수기, 뛰어오르기, 용의춤
pokemon215 = Pokemon("라프라스", ["물", "얼음"], -1, -1, 130, 85, 80, 85, 95, 60, [copy.deepcopy(skill23), copy.deepcopy(skill128), copy.deepcopy(skill89), copy.deepcopy(skill175)]) #파도타기, 얼음뭉치, 사념의박치기, 멸망의노래
pokemon216 = Pokemon("메타몽", ["노말"], -1, -1, 48, 48, 48, 48, 48, 48, [copy.deepcopy(skill223)]) #변신
pokemon217 = Pokemon("샤미드", ["물"], -1, -1, 130, 65, 60, 110, 95, 65, [copy.deepcopy(skill224), copy.deepcopy(skill117), copy.deepcopy(skill37), copy.deepcopy(skill16)]) #녹기, 열탕, 섀도볼, 시그널빔
pokemon218 = Pokemon("쥬피썬더", ["전기"], -1, -1, 65, 65, 60, 110, 95, 130, [copy.deepcopy(skill82), copy.deepcopy(skill24), copy.deepcopy(skill37), copy.deepcopy(skill196)]) #10만볼트, 방전, 섀도볼, 잠재파워
pokemon219 = Pokemon("부스터", ["불꽃"], -1, -1, 65, 130, 60, 95, 110, 65, [copy.deepcopy(skill63), copy.deepcopy(skill157), copy.deepcopy(skill225), copy.deepcopy(skill188)]) #플레어드라이브, 분연, 기가임팩트, 구멍파기
pokemon220 = Pokemon("암스타", ["바위", "물"], -1, -1, 70, 60, 125, 115, 70, 55, [copy.deepcopy(skill124), copy.deepcopy(skill23), copy.deepcopy(skill37), copy.deepcopy(skill118)]) #원시의힘, 파도타기, 냉동빔, 비바라기
pokemon221 = Pokemon("투구푸스", ["바위", "물"], -1, -1, 60, 115, 105, 65, 70, 80, [copy.deepcopy(skill71), copy.deepcopy(skill25), copy.deepcopy(skill29), copy.deepcopy(skill45)]) #스톤샤워, 폭포오르기, 깜짝베기, 깨트리기
pokemon222 = Pokemon("프테라", ["바위", "비행"], -1, -1, 80, 135, 85, 70, 95, 150, [copy.deepcopy(skill71), copy.deepcopy(skill79), copy.deepcopy(skill91), copy.deepcopy(skill20)]) #스톤샤워, 제비반환, 아이언헤드, 드래곤클로
pokemon223 = Pokemon("망나뇽", ["드래곤", "비행"], -1, -1, 91, 134, 95, 100, 100, 80, [copy.deepcopy(skill139), copy.deepcopy(skill11), copy.deepcopy(skill164), copy.deepcopy(skill20)]) #폭풍, 불꽃펀치, 아쿠아테일, 드래곤클로

pokemon_list = [pokemon1, pokemon2, pokemon3, pokemon4, pokemon5, pokemon6, pokemon7, pokemon8, pokemon9, pokemon10, pokemon11, pokemon12, pokemon13, pokemon14, pokemon15, pokemon16, pokemon17, pokemon18, pokemon19,
                pokemon20, pokemon21, pokemon22, pokemon23, pokemon24, pokemon25, pokemon26, pokemon27, pokemon28, pokemon29, pokemon30, pokemon31, pokemon32, pokemon33, pokemon34, pokemon35, pokemon36, pokemon37,
                pokemon38, pokemon39, pokemon40, pokemon41, pokemon42, pokemon43, pokemon44, pokemon45, pokemon46, pokemon47, pokemon48, pokemon49, pokemon50, pokemon51, pokemon52, pokemon53, pokemon54, pokemon55,
                pokemon56, pokemon57, pokemon58, pokemon59, pokemon60, pokemon61, pokemon62, pokemon63, pokemon64, pokemon65, pokemon66, pokemon67, pokemon68, pokemon69, pokemon70, pokemon71, pokemon72, pokemon73,
                pokemon74, pokemon75, pokemon76, pokemon77, pokemon78, pokemon79, pokemon80, pokemon81, pokemon82, pokemon83, pokemon84, pokemon85, pokemon86, pokemon87, pokemon88, pokemon89, pokemon90, pokemon91,
                pokemon92, pokemon93, pokemon94, pokemon95, pokemon96, pokemon97, pokemon98, pokemon99, pokemon100, pokemon101, pokemon102, pokemon103, pokemon104, pokemon105, pokemon106, pokemon107, pokemon108,
                pokemon109, pokemon110, pokemon111, pokemon112, pokemon113, pokemon114, pokemon115, pokemon116, pokemon117, pokemon118, pokemon119, pokemon120, pokemon121, pokemon122, pokemon123, pokemon124,
                pokemon125, pokemon126, pokemon127, pokemon128, pokemon129, pokemon130, pokemon131, pokemon132, pokemon133, pokemon134, pokemon135, pokemon136, pokemon137, pokemon138, pokemon139, pokemon140,
                pokemon141, pokemon142, pokemon143, pokemon144, pokemon145, pokemon146, pokemon147, pokemon148, pokemon149, pokemon150, pokemon151, pokemon152, pokemon153, pokemon154, pokemon155, pokemon156,
                pokemon157, pokemon158, pokemon159, pokemon160, pokemon161, pokemon162, pokemon163, pokemon164, pokemon165, pokemon166, pokemon167, pokemon168, pokemon169, pokemon170, pokemon171, pokemon172,
                pokemon173, pokemon174, pokemon175, pokemon176, pokemon177, pokemon178, pokemon179, pokemon180, pokemon181, pokemon182, pokemon183, pokemon184, pokemon185, pokemon186, pokemon187, pokemon188,
                pokemon189, pokemon190, pokemon191, pokemon192, pokemon193, pokemon194, pokemon195, pokemon196, pokemon197, pokemon198, pokemon199, pokemon200, pokemon201, pokemon202, pokemon203, pokemon204,
                pokemon205, pokemon206, pokemon207, pokemon208, pokemon209, pokemon210, pokemon211, pokemon212, pokemon213, pokemon214, pokemon215, pokemon216, pokemon217, pokemon218, pokemon219, pokemon220,
                pokemon221, pokemon222, pokemon223]
















In [ ]:

def lists_have_common_values(lst1, lst2):
    return len(set(lst1).intersection(set(lst2))) > 0

def combat(damaging : Pokemon, target : Pokemon, skill_index, weather):
    if(len(weather) > 1):
      if (weather[1] == -1):
          if ("햇살" in weather):
              if (PRINT_FLAG):
                  print(f"햇살이 약해졌다!")
          elif ("비" in weather):
              if (PRINT_FLAG):
                  print(f"비가 그쳤다!")
          elif ("눈" in weather):
              if (PRINT_FLAG):
                  print(f"눈이 그쳤다!")
          weather[0] = ""
    if (weather[0] == "햇살"):
        if (PRINT_FLAG):
            print(f"햇살이 강하다.")
    elif (weather[0] == "비"):
        if (PRINT_FLAG):
            print(f"비가 계속 내리고 있다.")
    elif (weather[0] == "눈"):
        if (PRINT_FLAG):
            print(f"눈이 계속 내리고 있다.")
        if ("얼음" not in damaging.types_list):
            damaging.health -= math.ceil(damaging.health / 16)
            if (PRINT_FLAG):
                print(f"싸라기눈이 {damaging.name}을(를) 덮쳤다! 현재 체력: {damaging.health if damaging.health >= 0 else 0}/{damaging.maxhealth}")
    damaging.state_control()
    if ("멸망" in damaging.state):
        return
    if (not lists_have_common_values(damaging.state, ["졸음2", "졸음3", "졸음4", "얼음", "전신마비", "자해", "풀죽음1", "풀죽음2", "반동1", "반동2"])):
        if ("발버둥" not in damaging.state):
            while (damaging.skill_list[skill_index].pp == 0):
                skill_index = random.randint(0, 3)
            target.take_damage(damaging.use_skill(skill_index, weather))
        else:
            if (PRINT_FLAG):
                print(f"{damaging.name}은(는) 더 이상 쓸 수 있는 기술이 없다!")
            target.take_damage(damaging.use_skill(-1, weather))



def coloring(p1: Pokemon, p2: Pokemon, turn, order):
    color = Fore.RED if order == "p1" else Fore.BLUE
    if PRINT_FLAG:
      print(color + '', end = "")
      print(f"\033[1m{turn}번째 턴 시작! {p1.name if order == 'p1' else p2.name}의 차례!\033[0;0m")

def rage_check(p, index):
    if (p.skill_list[index].pp == 0):
        for i in p.state:
            if (i in ["난동1", "난동2", "난동3"]):
                if ("혼란" in p.state):
                    i = "정상"
                else:
                    i = "혼란"

def pp_check(p):
    return p.skill_list[0].pp == 0 and p.skill_list[1].pp == 0 and p.skill_list[2].pp == 0 and p.skill_list[3].pp == 0

#주요함수
#기술 확률분포도 기준으로 기술선택
def choose_skill(my_pokemon, enemy_pokemon, random_index):
    t_probability = my_pokemon.skill_probabilities[transform_health_to_parts(my_pokemon)][transform_health_to_parts(enemy_pokemon)]

    # Create a copy of probabilities to avoid modifying the original
    modified_probability = t_probability[:]

    # Zero out probabilities for skills with zero PP
    for i in range(4):
        if my_pokemon.skill_list[i].pp <= 0:
            modified_probability[i] = 0

    # Calculate cumulative probabilities
    cumulative_prob = 0
    cumulative_probs = []
    for prob in modified_probability:
        cumulative_prob += prob
        cumulative_probs.append(cumulative_prob)

    # Choose skill based on random number
    random_number = random.uniform(0, cumulative_probs[-1])
    for i, prob in enumerate(cumulative_probs):
        if random_number < prob:
            random_index = i
            break

    return random_index


#공격 순서 정하는 알고리즘
#추후 수정 바람
def determine_attack_order(p1, p2, p1_random_index, p2_random_index):
    if p1.speed > p2.speed and p2.skill_list[p2_random_index].name not in Enum.PRIORITY_SKILL.value:
        return "p1", "p2"
    elif p2.speed >= p1.speed and p1.skill_list[p1_random_index].name not in Enum.PRIORITY_SKILL.value:
        return "p2", "p1"
    elif p1.skill_list[p1_random_index].name in Enum.PRIORITY_SKILL.value:
        return "p1", "p2"
    elif p2.skill_list[p2_random_index].name in Enum.PRIORITY_SKILL.value:
        return "p2", "p1"

#전투 실행 함수
def battle_turn(p1, p2, p1_random_index, p2_random_index, turn, order, weather):
    if order == "p1":
        coloring(p1, p2, turn, order)
        combat(p1, p2, p1_random_index, weather)
        if p1.health <= 0:
            if PRINT_FLAG:
              print(f"{p1.name}은(는) 쓰러졌다! ")
            return p2
        if p2.health <= 0:
            if PRINT_FLAG:
              print(f"{p2.name}은(는) 쓰러졌다! ")
            return p1
    elif order == "p2":
        coloring(p1, p2, turn, order)
        combat(p2, p1, p2_random_index, weather)
        if p2.health <= 0:
            if PRINT_FLAG:
              print(f"{p2.name}은(는) 쓰러졌다! ")
            return p1
        if p1.health <= 0:
            if PRINT_FLAG:
              print(f"{p1.name}은(는) 쓰러졌다! ")
            return p2
    return None
#체력변환함수
def transform_health_to_parts(pokemon: Pokemon):
  return int((pokemon.health / pokemon.maxhealth) * (NUMBER_OF_CASES_HP - 1))

In [ ]:
def save_battle_data(p1, p2, winner, p1_skill_count, p2_skill_count):
    #대충 저장하기
    with open('/content/battle_data.csv', 'a', newline='') as csvfile:
        fieldnames = ['Pokemon1', 'Pokemon2', 'Winner', 'P1_Skill1', 'P1_Skill2', 'P1_Skill3', 'P1_Skill4', 'P2_Skill1', 'P2_Skill2', 'P2_Skill3', 'P2_Skill4']
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames)

        if os.path.getsize('/content/battle_data.csv') == 0:
            writer.writeheader()

        writer.writerow({
            'Pokemon1': p1.name,
            'Pokemon2': p2.name,
            'Winner': winner.name if winner else '',
            'P1_Skill1': p1_skill_count[0],
            'P1_Skill2': p1_skill_count[1],
            'P1_Skill3': p1_skill_count[2],
            'P1_Skill4': p1_skill_count[3],
            'P2_Skill1': p2_skill_count[0],
            'P2_Skill2': p2_skill_count[1],
            'P2_Skill3': p2_skill_count[2],
            'P2_Skill4': p2_skill_count[3],
        })

def clear_csv_file(filename):
    open(filename, 'w').close()  # 초기화



In [ ]:

def battle1v1(p1: Pokemon, p2: Pokemon, log = False):
    p2_random_index = 0
    p1_random_index = 0
    p1_skill_count = np.zeros((NUMBER_OF_CASES_HP, NUMBER_OF_CASES_HP, NUM_OF_SKILL), dtype=int)  # 각 스킬의 사용 횟수를 저장할 리스트 초기화
    p2_skill_count = np.zeros((NUMBER_OF_CASES_HP, NUMBER_OF_CASES_HP, NUM_OF_SKILL), dtype=int)  # 각 스킬의 사용 횟수를 저장할 리스트 초기화
    p1_log = []
    p2_log = []
    # 게임 시작 메시지 출력
    if (PRINT_FLAG):
      print(Fore.RED + '', end = "")
      p1.display_info()
      print(Fore.BLACK + "VS")
      print(Fore.BLUE + '', end = "")
      p2.display_info()

    # 게임 진행 변수 초기화
    turn = 1
    weather = [""]
    weather_counter = 0

    while p1.health > 0 and p2.health > 0:
        old_weather = weather[0]

        if (pp_check(p1)):
            if "발버둥" not in p1.state:
                p1.state.append("발버둥")
        if (pp_check(p2)):
            if "발버둥" not in p2.state:
                p2.state.append("발버둥")

        # 날씨 상태 체크 및 출력
        if weather_counter == 0:
            if "햇살" in weather:
                if PRINT_FLAG:
                  print(f"햇살이 약해졌다!")
            elif "비" in weather:
                if PRINT_FLAG:
                  print(f"비가 그쳤다!")
            weather[0] = ""

        # Pokemon 1의 스킬 선택 및 인덱스 설정
        rage_check(p1, p1_random_index)
        p1_random_index = choose_skill(p1, p2, p1_random_index)
        # Pokemon 1의 스킬 사용 횟수 증가
        p1_skill_count[transform_health_to_parts(p1)][transform_health_to_parts(p2)][p1_random_index] += 1
        if (log):
          p1_log.append(p1.skill_list[p1_random_index].name)
        # Pokemon 2의 스킬 선택 및 인덱스 설정
        rage_check(p2, p2_random_index)
        p2_random_index = choose_skill(p2, p1, p2_random_index)
        # Pokemon 2의 스킬 사용 횟수 증가
        p2_skill_count[transform_health_to_parts(p2)][transform_health_to_parts(p1)][p2_random_index] += 1
        if (log):
          p2_log.append(p2.skill_list[p2_random_index].name)

        # 턴 진행 및 결과 확인
        p1_order, p2_order = determine_attack_order(p1, p2, p1_random_index, p2_random_index)

        winner = battle_turn(p1, p2, p1_random_index, p2_random_index,  turn, p1_order, weather)
        if winner is not None:
            if (log):
              print("\n" + p1.name, end = " ")
              for i in p1_log:
                print(i, end = " ")
              print("\n" + p2.name, end = " ")
              for i in p2_log:
                print(i, end = " ")
            #print(p1_skill_count)
            #save_battle_data(p1, p2, winner, p1_skill_count, p2_skill_count)
            if winner == p1:
                return p1, p2, p1_skill_count, p2_skill_count
            else:
                return p2, p1, p2_skill_count, p1_skill_count

        turn += 1
        winner = battle_turn(p1, p2, p1_random_index, p2_random_index,  turn, p2_order, weather)
        if winner is not None:
            if (log):
                print("\n" + p1.name, end = " ")
                for i in p1_log:
                  print(i, end = " ")
                print("\n" + p2.name, end = " ")
                for i in p2_log:
                  print(i, end = " ")
            #save_battle_data(p1, p2, winner, p1_skill_count, p2_skill_count)
            if winner == p1:
                return p1, p2, p1_skill_count, p2_skill_count
            else:
                return p2, p1, p2_skill_count, p1_skill_count

        turn += 1

        # 날씨 카운터 업데이트
        if old_weather == "" and weather[0] != "":
            weather_counter = 5
        if weather[0] != "":
            weather_counter -= 1
        if (p2.health <= 0):
          print("?")
          return p1, p2, p1_skill_count, p2_skill_count
        elif (p1.health <= 0):
          print("?")
          return p2, p1, p2_skill_count, p1_skill_count

    # # 무승부 메시지 출력
    # if turn > 500:
    #     print("무승부")
    print(turn)
    print(p1.health)
    print(p2.health)
    # # 종료 시 각 포켓몬의 스킬 사용 횟수 반환
    #return p1, p2, p1_skill_count, p2_skill_count
# if __name__ == '__main__':
#   temp1 = copy.deepcopy(pokemon1)
#   temp2 = copy.deepcopy(pokemon2)
#   PRINT_FLAG = True
#   winner, loser, _, _ =battle1v1(temp1, temp2)
#   print(winner.name)


In [ ]:
#실행하지 말것
#실수로도 누르지 말것
#재앙이 발생하는 버튼
# if __name__ == '__main__':
#     run_battles(5, pokemon1, pokemon2)

In [ ]:
#절대 실행하지 말껄
#리제로 경험하는 버튼
#clear_csv_file()

In [ ]:
def save_epoch_summary_to_csv(epoch, p):
    # Define the CSV file path
    csv_file_path = "/content/epoch_summary.csv"

    # Write the epoch summary to the CSV file
    with open(csv_file_path, mode = 'a', newline = '') as file:
        writer = csv.writer(file)
        if epoch == 0:
            header = ['Epoch'] + [f"{i}, {j}" for i in range(len(p.skill_probabilities)) for j in range(len(p.skill_probabilities[i]))]
            writer.writerow(header)
        flattened_probabilities = [item for sublist in p.skill_probabilities for item in sublist]

        # Write the epoch summary
        writer.writerow([f"Epoch {epoch + 1}"] + flattened_probabilities)


In [ ]:
def normalize_ratio(input_list):
    normalized_list = []
    for matrix in input_list:
        normalized_matrix = []
        for row in matrix:
            # 합산
            row_sum = sum(row)
            if row_sum == 0:
                row_sum = 1  # 나누기 0 대처하기

            # Normalize
            normalized_row = [x / row_sum for x in row]
            normalized_matrix.append(normalized_row)
        normalized_list.append(normalized_matrix)
    return normalized_list


In [ ]:
def get_list_depth(input_list):
    if (not isinstance(input_list, list) or len(input_list) == 0):
        # Base case: If the input is not a list, it's 0-dimensional
        return 0
    else:
        # Recursive case: Increment the depth for each nested list
        return 1 + get_list_depth(input_list[0])

In [ ]:
def update_skill_probabilities(skill_probabilities, skill_count, winner, lr = 0.001):
    # Convert skill_probabilities, skill_count, and winner to tensors
    skill_probabilities_tensor = torch.tensor(skill_probabilities, dtype = torch.float32)

    winner_tensor = torch.tensor(winner, dtype = torch.float32)

    # Reshape skill_probabilities for calculation
    flat_skill_probabilities = skill_probabilities_tensor.view(-1, NUM_OF_SKILL)

    flat_winner = winner_tensor.view(-1)
    # Update probabilities
    for e in range(len(flat_winner)):
        skill_count_tensor = torch.tensor(skill_count[e], dtype=torch.float32)
        flat_skill_count = skill_count_tensor.view(-1, NUM_OF_SKILL)
        for i in range(NUM_OF_SKILL):
            # Calculate the update based on winner and skill_count
            update = lr * (1 if winner[e] else -1) * flat_skill_count[:, i]
            # Apply update
            flat_skill_probabilities[:, i] += update

    # Clamp probabilities to [0.01, 0.99] range
    flat_skill_probabilities.clamp_(0.01, 0.99)

    # Reshape the updated probabilities
    result_prob = flat_skill_probabilities.view(NUMBER_OF_CASES_HP, NUMBER_OF_CASES_HP, NUM_OF_SKILL).tolist()
    result_prob = normalize_ratio(result_prob)

    return result_prob

In [ ]:
def volume_to_images(data_list):
    data = np.array(data_list)
    # Calculate the dimensions of the volume
    side_length = int(np.ceil(np.sqrt(data.size)))

    # Reshape the array into a square
    square_array = np.resize(data, (side_length, side_length))


    # Convert the resized array back to a list
    resized_list = square_array.tolist()

    # Create and display the image
    image = Image.new("L", (len(resized_list[0]), len(resized_list)))
    image.putdata([(int(pixel * 255)) for row in resized_list for pixel in row])
    original_width, original_height = image.size

    # Double the size
    new_width = original_width * 4
    new_height = original_height * 4

    # Resize the image
    resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    display(resized_image)


In [ ]:
def adversarial_learning(p1, p2, epochs, batch_size):
    #p2.skill_probabilities = my_list = [[0.1, 1, 0.1, 0.1] for _ in range(16)]
    temp_p1 = copy.deepcopy(p1)
    temp_p2 = copy.deepcopy(p2)
    p1_epoch_skill_count = []
    p2_epoch_skill_count = []
    p1_epoch_wins = []
    p2_epoch_wins = []
    c = 0
    for epoch in range(epochs):
        p1_win = 0
        p2_win = 0

        p1_prob = update_skill_probabilities(temp_p1.skill_probabilities, p1_epoch_skill_count, p1_epoch_wins)
        p2_prob = update_skill_probabilities(temp_p2.skill_probabilities, p2_epoch_skill_count, p2_epoch_wins)
        p1_epoch_skill_count = []
        p2_epoch_skill_count = []
        p1_epoch_wins = []
        p2_epoch_wins = []
        for _ in range(batch_size):
            temp_p1.reset_attributes()
            temp_p2.reset_attributes()
            temp_p1.skill_probabilities = p1_prob
            temp_p2.skill_probabilities = p2_prob
            winner, _, skill_count_p1, skill_count_p2 = battle1v1(temp_p1, temp_p2)
            p1_epoch_skill_count.append(skill_count_p1)
            p2_epoch_skill_count.append(skill_count_p2)
            if winner == temp_p1:
              p1_epoch_wins.append(True)
              p2_epoch_wins.append(False)
              p1_win += 1
            else:
              p1_epoch_wins.append(False)
              p2_epoch_wins.append(True)
              p2_win +=1

            c+=1

        # Print epoch summary
        print(f"Epoch {epoch + 1}")
        print(f"{p1.name} Probabilities: {temp_p1.skill_probabilities}, {p2.name}") #Probabilities: {temp_p2.skill_probabilities}")
        print(f"{p1.name} Wins: {p1_win}, {p2.name} Wins: {p2_win}")
        #save_epoch_summary_to_csv(epoch, temp_p1)
    volume_to_images(temp_p1.skill_probabilities)
    volume_to_images(temp_p2.skill_probabilities)
    for i in range(10):
      temp_p1.reset_attributes()
      temp_p2.reset_attributes()
      battle1v1(temp_p1, temp_p2, True)

In [ ]:
# Run adversarial learning
# "/content/epoch_summary.csv"
#adversarial_learning(copy.deepcopy(pokemon36), copy.deepcopy(pokemon72), epochs = 50, batch_size = 100)

In [ ]:
cProfile.run('adversarial_learning(copy.deepcopy(pokemon1), copy.deepcopy(pokemon32), epochs = 200, batch_size = 1000)', filename='profile_stats')

# Load the profiling statistics
stats = pstats.Stats('profile_stats')
stats.sort_stats('tottime')
stats.print_stats()

In [15]:
!git add Pokeman.ipynb
!git commit -m "Init"

fatal: pathspec 'Pokeman.ipynb' did not match any files
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [ ]:
!git push -u origin main